## Neural Network

Here is the code for our neural network.

To load the information on the models we trained, run the following cell:

In [1]:
import numpy as np
from sklearn.neural_network import MLPClassifier
import pickle # to load information from trained models

models_file = open('models', 'rb')

models = pickle.load(models_file)

models_file.close()

`models` has the following structure:
```python
models = [
    model_info_a_log,  # models[0]
    model_info_b_log,  # models[1]
    model_info_c_log,  # models[2]
    model_info_d_log,  # models[3]
    
    model_info_a_tanh, # models[4]
    model_info_b_tanh, # models[5]
    model_info_c_tanh, # models[6]
    model_info_d_tanh  # models[7]
]
```

where each `model` index holds a nested dictionary of model information, keyed using the training and test accuracies:

```python
# structure of models[i]
models[i] = j : { # key is a number
                    'features' : name,
                    'layers' : layers,
                    'test size' : test_size,
                    'alpha' : alpha,
                    'lambda' : lam,
                    'train accuracy' : tr_acc,
                    'test accuracy' : ts_acc #,
                    # 'classifier' : clf # optional, adds ~2GB to models file so it was not saved 
                }

{
    # each key is a tuple of the training and testing accuracies
    (training_accuracy, testing_accuracy) : {
            # value keys are for name, layers, test size, alpha value, and lambda value
            'name' : name,
            'layers' : layers,
            'test size' : test_size,
            'alpha' : alpha,
            'lambda' : lam
        }
}
```

In [6]:
models

[{0: {'features': 'all features',
   'layers': (112, 56, 30, 10),
   'test size': 0.4,
   'alpha': 1.0,
   'lambda': 1000,
   'train accuracy': 0.30787037037037035,
   'test accuracy': 0.3151041666666667},
  1: {'features': 'all features',
   'layers': (112, 56, 30, 10),
   'test size': 0.4,
   'alpha': 1.0,
   'lambda': 100,
   'train accuracy': 0.34664351851851855,
   'test accuracy': 0.3133680555555556},
  2: {'features': 'all features',
   'layers': (112, 56, 30, 10),
   'test size': 0.4,
   'alpha': 1.0,
   'lambda': 1.0,
   'train accuracy': 0.33622685185185186,
   'test accuracy': 0.3289930555555556},
  3: {'features': 'all features',
   'layers': (112, 56, 30, 10),
   'test size': 0.4,
   'alpha': 1.0,
   'lambda': 0.1,
   'train accuracy': 0.3402777777777778,
   'test accuracy': 0.3229166666666667},
  4: {'features': 'all features',
   'layers': (112, 56, 30, 10),
   'test size': 0.4,
   'alpha': 1.0,
   'lambda': 0.01,
   'train accuracy': 0.33738425925925924,
   'test accura

In [8]:
for m in models:
    print(len(m))

576
576
576
576
576
576
576
576


=================================================================

There are 4 csv files corresponding to the different users' datasets:

In [7]:
import numpy as np
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

df1 = pd.read_csv('./Dataset/user_a.csv', delimiter=',')
df2 = pd.read_csv('./Dataset/user_b.csv', delimiter=',')
df3 = pd.read_csv('./Dataset/user_c.csv', delimiter=',')
df4 = pd.read_csv('./Dataset/user_d.csv', delimiter=',')

In [8]:
from sklearn.preprocessing import StandardScaler  # It is important in neural networks to scale the date
from sklearn.model_selection import train_test_split  # The standard - train/test to prevent overfitting and choose hyperparameters
from sklearn.metrics import confusion_matrix, accuracy_score # 
import numpy as np
from sklearn.neural_network import MLPClassifier

# print training information -- accuracy, confusion matrix, sample predictions
def print_training_info(test_acc, train_acc, X_test, y_test, clf):
    print("* * *")
    print(f"training accuracy: {train_acc*100}%")
    print(f"testing accuracy: {test_acc*100}%")

    # show confusion matrix
    y_pred = clf.predict(X_test)
    cm = confusion_matrix(y_test, y_pred)
    print (f"\nConfusion Matrix: \n{cm}")

    print("\nfirst 10 test values and predictions:")

    # show predictions of first 10 test examples
    n = 10
    y_pred_n = clf.predict_proba(X_test[:n, :])
    for i in range(n):
        print(y_test[i], y_pred_n[i])

## Setting up for training

Below shows how we train the models. 

The `run` function trains a model according to its input specifications:

- It first splits our provided test data as specified using `test_size`.
- then it uses SKLearn's `MLPClassifier` to create a neural net with the desired number of layers, the max number of iterations, and the alpha and lambda values.
- it trains the neural net using the test data, then finds its accuracy on the training data and the test data.

In [9]:
def run(X, y, layers, test_size=0.4, lam=0.001, a=0.25, iter_num=3000, batch_size='auto', act='logistic'):
    X_scale = StandardScaler()
    X = X_scale.fit_transform(X)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)
    clf = MLPClassifier(activation=act,
                        hidden_layer_sizes=layers, 
                        alpha=lam, # refers to our lambda
                        learning_rate_init=a, # refers to our alpha
                        max_iter=iter_num,
                        batch_size=batch_size, # default is 200
                        solver='sgd', 
                        random_state=1,
                        verbose=False
                       )
    clf.fit(X_train, y_train)
    
    train_acc = clf.score(X_train, y_train)
    test_acc = clf.score(X_test, y_test)

    # print accuracy, confusion matrix, sample predictions above threshold
    if test_acc > 0.8:
        print_training_info(test_acc, train_acc, X_test, y_test, clf)
        
    return train_acc, test_acc, clf

the `automate_run` function calls run many times using different specifications. Here is a list of the things `automate_run` varies so far:
- the features used (all, just the stds, just the means)
- the number of hidden layers and their structure
- the activation function (logistic or tanh)
- the training size/test size
- the alpha values ($\alpha \in [0.001, 1]$)
- the lambda values ($\lambda \in [0.00001, 1000]$)


In [10]:
# goes through many different ways to train a model
def automate_run(df, dic, act='logistic'):
    i = 0
    for name, X, y in (('all features', df.values[:, 1:], df.values[:, 0].astype(int)), 
                       ('std features', df.values[:, 1::2], df.values[:, 0].astype(int)), 
                       ('mean features', df.values[:, 2::2], df.values[:, 0].astype(int))):
        for layers in ((112, 56, 30, 10),
                       (112),
                       (112, 112, 112)):
            for test_size in (0.4, 0.3):
                for alpha in (1., 0.1, 0.01, 0.001):
                    for lam in (1000, 100, 1., 0.1, 0.01, 0.001, 0.0001, 0.00001):
                        print('\n==============================\n')
                        print(f'{name}:\nlayers={layers}\ntest_size={test_size}\nalpha={alpha}\nlambda={lam}\n')
                        tr_acc, ts_acc, clf = run(X, y, layers, test_size, lam, alpha, act=act)
                        dic[i] = {
                            'features' : name,
                            'layers' : layers,
                            'test size' : test_size,
                            'alpha' : alpha,
                            'lambda' : lam,
                            'train accuracy' : tr_acc,
                            'test accuracy' : ts_acc,
                            'classifier' : clf
                        }
                        i += 1

In [18]:
# prints the model accuracies and info above the given accuracy threshold
def show_above(d, accuracy=0.85):
    to_print = []
    for key, info in d.items():
        if info['test accuracy'] > accuracy:
            to_print.append((info['test accuracy'], key, info))

    for acc, key, info in sorted(to_print, reverse=True):
        print(acc)
        for k, v in info.items():
            print(f'\t{k}: {v}')

## Training the models on different users

### User A

In [12]:
model_info_a_log = {} # key is just an int, values are info
automate_run(df1, model_info_a_log)



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=1.0
lambda=1000



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=1.0
lambda=100



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=1.0
lambda=1.0



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=1.0
lambda=0.1



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=1.0
lambda=0.01



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=1.0
lambda=0.001



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=1.0
lambda=0.0001



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=1.0
lambda=1e-05



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=0.1
lambda=1000



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=0.1
lambda=100



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=0.1
lambda=1.0



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=0.1
lambda=0.1



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=0.1
l

* * *
training accuracy: 100.0%
testing accuracy: 81.25%

Confusion Matrix: 
[[308  48  25]
 [ 35 289  42]
 [ 21  45 339]]

first 10 test values and predictions:
2 [5.59632257e-12 2.81061611e-04 9.99718938e-01]
1 [9.71922081e-01 2.73190701e-02 7.58849159e-04]
1 [0.00243584 0.72025404 0.27731013]
2 [0.04702062 0.86324477 0.08973461]
2 [1.23427584e-03 2.87277650e-04 9.98478447e-01]
0 [9.42198118e-01 5.75056082e-02 2.96274147e-04]
2 [5.05479900e-07 7.31945082e-04 9.99267549e-01]
1 [0.00486078 0.78392056 0.21121866]
0 [9.99999970e-01 3.00095726e-08 7.93475759e-12]
0 [9.99719298e-01 9.26453684e-08 2.80609003e-04]


all features:
layers=112
test_size=0.4
alpha=1.0
lambda=0.0001

* * *
training accuracy: 100.0%
testing accuracy: 80.55555555555556%

Confusion Matrix: 
[[312  33  35]
 [ 33 328  34]
 [ 28  61 288]]

first 10 test values and predictions:
1 [6.97603841e-04 9.99301382e-01 1.01427599e-06]
1 [9.32755676e-03 9.90668856e-01 3.58725373e-06]
0 [0.79465419 0.00325112 0.20209469]
0 [9.9999

* * *
training accuracy: 100.0%
testing accuracy: 81.59722222222221%

Confusion Matrix: 
[[311  51  30]
 [ 21 315  49]
 [ 19  42 314]]

first 10 test values and predictions:
1 [0.01363517 0.97671179 0.00965305]
0 [0.95307772 0.04472091 0.00220137]
1 [2.02544702e-07 9.99999627e-01 1.70031333e-07]
0 [9.97865976e-01 6.64169753e-04 1.46985378e-03]
1 [0.00751149 0.97717652 0.015312  ]
2 [0.00143423 0.045116   0.95344978]
0 [9.99386903e-01 3.56577793e-04 2.56519210e-04]
1 [6.71921829e-04 9.99317993e-01 1.00848407e-05]
0 [9.97488401e-01 2.33460746e-03 1.76991146e-04]
0 [0.98376901 0.01408783 0.00214317]


all features:
layers=112
test_size=0.4
alpha=0.01
lambda=1e-05

* * *
training accuracy: 100.0%
testing accuracy: 80.03472222222221%

Confusion Matrix: 
[[312  26  32]
 [ 37 299  42]
 [ 27  66 311]]

first 10 test values and predictions:
0 [9.79867121e-01 6.21848586e-04 1.95110300e-02]
2 [0.0135557  0.00220502 0.98423927]
1 [0.0063559  0.99045678 0.00318732]
0 [0.81990366 0.17404581 0.006050

C:\Users\Elijah\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:684: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (3000) reached and the optimization hasn't converged yet.
  warnings.warn(




all features:
layers=112
test_size=0.4
alpha=0.001
lambda=0.01



...



std features:
layers=112
test_size=0.3
alpha=0.001
lambda=0.0001



std features:
layers=112
test_size=0.3
alpha=0.001
lambda=1e-05



std features:
layers=(112, 112, 112)
test_size=0.4
alpha=1.0
lambda=1000



std features:
layers=(112, 112, 112)
test_size=0.4
alpha=1.0
lambda=100



std features:
layers=(112, 112, 112)
test_size=0.4
alpha=1.0
lambda=1.0



std features:
layers=(112, 112, 112)
test_size=0.4
alpha=1.0
lambda=0.1



std features:
layers=(112, 112, 112)
test_size=0.4
alpha=1.0
lambda=0.01



std features:
layers=(112, 112, 112)
test_size=0.4
alpha=1.0
lambda=0.001



std features:
layers=(112, 112, 112)
test_size=0.4
alpha=1.0
lambda=0.0001



std features:
layers=(112, 112, 112)
test_size=0.4
alpha=1.0
lambda=1e-05



std features:
layers=(112, 112, 112)
test_size=0.4
alpha=0.1
lambda=1000



std features:
layers=(112, 112, 112)
test_size=0.4
alpha=0.1
lambda=100



std features:
layers=(112, 112, 112)
test_size=0.4
alpha=0.1
lambda=1.0



std features:
layers=(112, 1



mean features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=0.1
lambda=1.0



mean features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=0.1
lambda=0.1



mean features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=0.1
lambda=0.01



mean features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=0.1
lambda=0.001



mean features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=0.1
lambda=0.0001



mean features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=0.1
lambda=1e-05



mean features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=0.01
lambda=1000



mean features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=0.01
lambda=100



mean features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=0.01
lambda=1.0



mean features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=0.01
lambda=0.1



mean features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=0.01
lambda=0.01



mean features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=0.01
lambda=0.001



mean features:
layers=(112, 56, 30, 10)
test



mean features:
layers=112
test_size=0.4
alpha=0.1
lambda=0.1



mean features:
layers=112
test_size=0.4
alpha=0.1
lambda=0.01



mean features:
layers=112
test_size=0.4
alpha=0.1
lambda=0.001



mean features:
layers=112
test_size=0.4
alpha=0.1
lambda=0.0001



mean features:
layers=112
test_size=0.4
alpha=0.1
lambda=1e-05



mean features:
layers=112
test_size=0.4
alpha=0.01
lambda=1000



mean features:
layers=112
test_size=0.4
alpha=0.01
lambda=100



mean features:
layers=112
test_size=0.4
alpha=0.01
lambda=1.0



mean features:
layers=112
test_size=0.4
alpha=0.01
lambda=0.1



mean features:
layers=112
test_size=0.4
alpha=0.01
lambda=0.01

* * *
training accuracy: 99.82638888888889%
testing accuracy: 80.03472222222221%

Confusion Matrix: 
[[317  41  27]
 [ 37 295  48]
 [ 41  36 310]]

first 10 test values and predictions:
1 [6.51484133e-05 9.90424686e-01 9.51016562e-03]
0 [9.99493329e-01 4.95842877e-04 1.08285316e-05]
0 [9.82699586e-01 1.66676454e-04 1.71337374e-02]
1 [3.2461807

* * *
training accuracy: 99.95039682539682%
testing accuracy: 82.63888888888889%

Confusion Matrix: 
[[266  20  13]
 [ 27 217  40]
 [ 20  30 231]]

first 10 test values and predictions:
0 [9.99944106e-01 1.57839103e-05 4.01100570e-05]
0 [0.78669287 0.20637908 0.00692805]
0 [0.90028709 0.06725138 0.03246152]
0 [0.07101405 0.00094171 0.92804424]
1 [0.07625464 0.83375265 0.08999271]
1 [7.60444055e-03 9.92089457e-01 3.06102591e-04]
1 [8.94666960e-04 9.96267732e-01 2.83760081e-03]
1 [0.02923693 0.01670729 0.95405579]
0 [9.80138742e-01 2.46594358e-04 1.96146638e-02]
2 [0.01904407 0.92260815 0.05834778]


mean features:
layers=112
test_size=0.3
alpha=0.01
lambda=1e-05



mean features:
layers=112
test_size=0.3
alpha=0.001
lambda=1000



mean features:
layers=112
test_size=0.3
alpha=0.001
lambda=100



mean features:
layers=112
test_size=0.3
alpha=0.001
lambda=1.0



mean features:
layers=112
test_size=0.3
alpha=0.001
lambda=0.1



mean features:
layers=112
test_size=0.3
alpha=0.001
lambda=0.0



mean features:
layers=(112, 112, 112)
test_size=0.3
alpha=0.001
lambda=0.0001



mean features:
layers=(112, 112, 112)
test_size=0.3
alpha=0.001
lambda=1e-05



In [19]:
# looking at the top models
show_above(model_info_a_log, 0.85)

0.8645833333333334
	features: all features
	layers: 112
	test size: 0.3
	alpha: 1.0
	lambda: 0.01
	train accuracy: 1.0
	test accuracy: 0.8645833333333334
	classifier: MLPClassifier(activation='logistic', alpha=0.01, hidden_layer_sizes=112,
              learning_rate_init=1.0, max_iter=3000, random_state=1,
              solver='sgd')
0.8611111111111112
	features: all features
	layers: 112
	test size: 0.3
	alpha: 0.1
	lambda: 1e-05
	train accuracy: 1.0
	test accuracy: 0.8611111111111112
	classifier: MLPClassifier(activation='logistic', alpha=1e-05, hidden_layer_sizes=112,
              learning_rate_init=0.1, max_iter=3000, random_state=1,
              solver='sgd')
0.8611111111111112
	features: all features
	layers: 112
	test size: 0.3
	alpha: 1.0
	lambda: 0.0001
	train accuracy: 1.0
	test accuracy: 0.8611111111111112
	classifier: MLPClassifier(activation='logistic', hidden_layer_sizes=112,
              learning_rate_init=1.0, max_iter=3000, random_state=1,
              solver='sgd

In [21]:
# Using tanh
model_info_a_tanh = {}
automate_run(df1, model_info_a_tanh, act='tanh')



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=1.0
lambda=1000



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=1.0
lambda=100



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=1.0
lambda=1.0



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=1.0
lambda=0.1



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=1.0
lambda=0.01



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=1.0
lambda=0.001



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=1.0
lambda=0.0001



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=1.0
lambda=1e-05



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=0.1
lambda=1000



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=0.1
lambda=100



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=0.1
lambda=1.0

* * *
training accuracy: 99.65277777777779%
testing accuracy: 83.76736111111111%

Confusion Matrix: 
[[315  25  19]
 [ 34 317  43]
 [ 23  43 33



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=0.001
lambda=0.01



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=0.001
lambda=0.001



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=0.001
lambda=0.0001



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=0.001
lambda=1e-05



all features:
layers=(112, 56, 30, 10)
test_size=0.3
alpha=1.0
lambda=1000



all features:
layers=(112, 56, 30, 10)
test_size=0.3
alpha=1.0
lambda=100



all features:
layers=(112, 56, 30, 10)
test_size=0.3
alpha=1.0
lambda=1.0



all features:
layers=(112, 56, 30, 10)
test_size=0.3
alpha=1.0
lambda=0.1



all features:
layers=(112, 56, 30, 10)
test_size=0.3
alpha=1.0
lambda=0.01



all features:
layers=(112, 56, 30, 10)
test_size=0.3
alpha=1.0
lambda=0.001



all features:
layers=(112, 56, 30, 10)
test_size=0.3
alpha=1.0
lambda=0.0001



all features:
layers=(112, 56, 30, 10)
test_size=0.3
alpha=1.0
lambda=1e-05



all features:
layers=(112, 56, 30, 10)
test_size=

* * *
training accuracy: 100.0%
testing accuracy: 82.06018518518519%

Confusion Matrix: 
[[250  17  24]
 [ 26 240  35]
 [ 18  35 219]]

first 10 test values and predictions:
2 [0.00605567 0.11202648 0.88191785]
0 [9.99785769e-01 3.87222566e-05 1.75508263e-04]
1 [2.20377603e-05 9.99839362e-01 1.38600003e-04]
1 [0.01071982 0.98434247 0.00493772]
0 [0.70208764 0.27484057 0.02307178]
1 [0.00827928 0.73491631 0.25680441]
0 [9.97034206e-01 2.94411713e-03 2.16772274e-05]
1 [0.00814826 0.99054644 0.0013053 ]
0 [0.62513665 0.07189757 0.30296578]
1 [1.39657378e-03 9.97638936e-01 9.64490154e-04]


all features:
layers=(112, 56, 30, 10)
test_size=0.3
alpha=0.01
lambda=1e-05



all features:
layers=(112, 56, 30, 10)
test_size=0.3
alpha=0.001
lambda=1000



all features:
layers=(112, 56, 30, 10)
test_size=0.3
alpha=0.001
lambda=100



all features:
layers=(112, 56, 30, 10)
test_size=0.3
alpha=0.001
lambda=1.0

* * *
training accuracy: 100.0%
testing accuracy: 85.18518518518519%

Confusion Matrix: 
[

* * *
training accuracy: 100.0%
testing accuracy: 81.42361111111111%

Confusion Matrix: 
[[304  38  25]
 [ 32 313  49]
 [ 21  49 321]]

first 10 test values and predictions:
1 [1.70164368e-03 9.98289482e-01 8.87453165e-06]
2 [1.17092852e-04 8.54852112e-04 9.99028055e-01]
2 [8.02248762e-02 7.21725537e-04 9.19053398e-01]
0 [9.98871475e-01 3.24388330e-04 8.04136171e-04]
1 [0.06900206 0.19065321 0.74034473]
2 [7.93228273e-06 6.77738892e-05 9.99924294e-01]
1 [6.43437078e-06 9.99779959e-01 2.13606470e-04]
0 [9.99829215e-01 6.33566953e-05 1.07428406e-04]
1 [5.59915620e-04 9.93602174e-01 5.83791052e-03]
2 [0.23254925 0.00250261 0.76494815]


all features:
layers=112
test_size=0.4
alpha=0.01
lambda=1000



all features:
layers=112
test_size=0.4
alpha=0.01
lambda=100



all features:
layers=112
test_size=0.4
alpha=0.01
lambda=1.0

* * *
training accuracy: 99.82638888888889%
testing accuracy: 84.375%

Confusion Matrix: 
[[328  36  17]
 [ 23 313  45]
 [ 21  38 331]]

first 10 test values and predi



all features:
layers=112
test_size=0.3
alpha=1.0
lambda=0.1



all features:
layers=112
test_size=0.3
alpha=1.0
lambda=0.01



all features:
layers=112
test_size=0.3
alpha=1.0
lambda=0.001



all features:
layers=112
test_size=0.3
alpha=1.0
lambda=0.0001



all features:
layers=112
test_size=0.3
alpha=1.0
lambda=1e-05



all features:
layers=112
test_size=0.3
alpha=0.1
lambda=1000



all features:
layers=112
test_size=0.3
alpha=0.1
lambda=100



all features:
layers=112
test_size=0.3
alpha=0.1
lambda=1.0

* * *
training accuracy: 92.26190476190477%
testing accuracy: 83.2175925925926%

Confusion Matrix: 
[[268  19  11]
 [ 32 234  27]
 [ 31  25 217]]

first 10 test values and predictions:
1 [0.05746494 0.87111273 0.07142234]
0 [0.83741362 0.06651186 0.09607452]
1 [0.00134144 0.97517647 0.0234821 ]
0 [0.58385531 0.27517697 0.14096772]
0 [0.72554111 0.16137459 0.1130843 ]
1 [0.11636267 0.88248785 0.00114948]
0 [9.80929169e-01 1.85066020e-02 5.64229012e-04]
1 [0.02213023 0.77522407 0.2026

...



mean features:
layers=(112, 56, 30, 10)
test_size=0.3
alpha=0.01
lambda=0.01

* * *
training accuracy: 100.0%
testing accuracy: 83.44907407407408%

Confusion Matrix: 
[[240  20  19]
 [ 23 231  32]
 [ 17  32 250]]

first 10 test values and predictions:
0 [9.99904068e-01 1.03854336e-05 8.55464937e-05]
1 [0.00155936 0.05722796 0.94121268]
1 [4.51097765e-06 9.99180358e-01 8.15131321e-04]
1 [9.98832490e-01 1.10313687e-03 6.43735681e-05]
0 [0.36012035 0.35824759 0.28163206]
1 [5.74713824e-06 9.99669264e-01 3.24988837e-04]
1 [3.61358636e-04 9.99637341e-01 1.30062576e-06]
1 [9.98702806e-01 4.52386719e-04 8.44807570e-04]
0 [0.94173008 0.00645509 0.05181484]
1 [1.08127511e-05 9.98347285e-01 1.64190220e-03]


mean features:
layers=(112, 56, 30, 10)
test_size=0.3
alpha=0.01
lambda=0.001

* * *
training accuracy: 100.0%
testing accuracy: 81.82870370370371%

Confusion Matrix: 
[[246  22  18]
 [ 35 238  21]
 [ 24  37 223]]

first 10 test values and predictions:
2 [4.77301533e-04 2.63728654e-02 9.73

* * *
training accuracy: 100.0%
testing accuracy: 80.12152777777779%

Confusion Matrix: 
[[316  32  42]
 [ 36 312  44]
 [ 24  51 295]]

first 10 test values and predictions:
0 [0.212883   0.77530707 0.01180993]
1 [7.46952915e-06 9.99928920e-01 6.36105516e-05]
1 [0.02596672 0.96985942 0.00417386]
1 [0.33860574 0.66053121 0.00086305]
2 [0.00652976 0.00466215 0.98880809]
1 [0.38346653 0.57974331 0.03679016]
1 [1.58507580e-03 9.97562450e-01 8.52474359e-04]
1 [0.13174179 0.85469341 0.0135648 ]
2 [0.13665712 0.55288216 0.31046072]
0 [0.00991566 0.98676025 0.00332409]


mean features:
layers=112
test_size=0.4
alpha=0.01
lambda=0.01



mean features:
layers=112
test_size=0.4
alpha=0.01
lambda=0.001

* * *
training accuracy: 100.0%
testing accuracy: 82.8125%

Confusion Matrix: 
[[322  29  30]
 [ 40 313  35]
 [ 27  37 319]]

first 10 test values and predictions:
1 [0.07414565 0.92005535 0.005799  ]
2 [3.54585950e-05 8.93866511e-02 9.10577890e-01]
2 [1.04998875e-05 5.26411511e-01 4.73577989e-01]




mean features:
layers=112
test_size=0.3
alpha=0.01
lambda=1.0



mean features:
layers=112
test_size=0.3
alpha=0.01
lambda=0.1

* * *
training accuracy: 99.95039682539682%
testing accuracy: 81.59722222222221%

Confusion Matrix: 
[[263  26  24]
 [ 21 217  36]
 [ 11  41 225]]

first 10 test values and predictions:
0 [0.91099247 0.07900924 0.00999829]
2 [4.12046738e-03 5.30274241e-05 9.95826505e-01]
0 [0.66817634 0.31663113 0.01519253]
0 [0.92555686 0.02642314 0.04802001]
0 [9.94189362e-01 5.23557819e-03 5.75060131e-04]
2 [0.1380524  0.02971366 0.83223394]
2 [0.00520597 0.02270771 0.97208633]
1 [3.72411169e-04 8.01815622e-01 1.97811966e-01]
1 [0.11597599 0.64359161 0.2404324 ]
2 [4.30609740e-04 1.56222222e-05 9.99553768e-01]


mean features:
layers=112
test_size=0.3
alpha=0.01
lambda=0.01

* * *
training accuracy: 100.0%
testing accuracy: 80.55555555555556%

Confusion Matrix: 
[[252  19  31]
 [ 27 228  44]
 [ 17  30 216]]

first 10 test values and predictions:
1 [1.37676295e-03 9.986121

* * *
training accuracy: 100.0%
testing accuracy: 80.55555555555556%

Confusion Matrix: 
[[316  25  35]
 [ 37 304  49]
 [ 30  48 308]]

first 10 test values and predictions:
2 [0.15149561 0.08805863 0.76044577]
0 [9.99601434e-01 3.28874006e-04 6.96921376e-05]
2 [8.90226636e-06 4.84563714e-03 9.95145461e-01]
1 [0.0739252  0.0043049  0.92176989]
0 [9.9595820e-01 3.5466592e-06 4.0382534e-03]
1 [6.92315950e-04 8.98302897e-01 1.01004787e-01]
1 [0.29964125 0.46883846 0.23152029]
2 [2.66272212e-07 9.99945791e-01 5.39427861e-05]
1 [4.57889219e-04 9.99518092e-01 2.40184418e-05]
0 [9.99005775e-01 2.41721418e-06 9.91807470e-04]


mean features:
layers=(112, 112, 112)
test_size=0.4
alpha=0.1
lambda=1e-05

* * *
training accuracy: 100.0%
testing accuracy: 81.68402777777779%

Confusion Matrix: 
[[325  34  26]
 [ 31 311  47]
 [ 24  49 305]]

first 10 test values and predictions:
0 [9.96168985e-01 3.83098332e-03 3.13909156e-08]
0 [9.99998943e-01 1.05727545e-06 2.73488134e-11]
0 [9.99216608e-01 1.46230



mean features:
layers=(112, 112, 112)
test_size=0.3
alpha=0.01
lambda=100



mean features:
layers=(112, 112, 112)
test_size=0.3
alpha=0.01
lambda=1.0

* * *
training accuracy: 98.61111111111111%
testing accuracy: 80.78703703703704%

Confusion Matrix: 
[[245  32  21]
 [ 23 225  38]
 [ 18  34 228]]

first 10 test values and predictions:
1 [1.00852393e-06 9.98963765e-01 1.03522613e-03]
0 [9.96295738e-01 3.62231152e-03 8.19505540e-05]
1 [1.23621751e-05 9.99185508e-01 8.02130271e-04]
2 [0.03735363 0.13424292 0.82840345]
0 [5.17389943e-01 4.82554606e-01 5.54510475e-05]
1 [0.00142745 0.87266987 0.12590269]
2 [0.00947271 0.26475446 0.72577283]
1 [0.04944855 0.91944317 0.03110828]
1 [3.42584980e-04 9.60745104e-01 3.89123115e-02]
0 [0.60986504 0.01591158 0.37422338]


mean features:
layers=(112, 112, 112)
test_size=0.3
alpha=0.01
lambda=0.1

* * *
training accuracy: 100.0%
testing accuracy: 82.87037037037037%

Confusion Matrix: 
[[264  19  18]
 [ 24 236  25]
 [ 22  40 216]]

first 10 test val

In [22]:
show_above(model_info_a_tanh, 0.85)

0.8703703703703703
	features: all features
	layers: 112
	test size: 0.3
	alpha: 0.1
	lambda: 0.1
	train accuracy: 1.0
	test accuracy: 0.8703703703703703
	classifier: MLPClassifier(activation='tanh', alpha=0.1, hidden_layer_sizes=112,
              learning_rate_init=0.1, max_iter=3000, random_state=1,
              solver='sgd')
0.8692129629629629
	features: all features
	layers: (112, 112, 112)
	test size: 0.3
	alpha: 0.1
	lambda: 0.001
	train accuracy: 1.0
	test accuracy: 0.8692129629629629
	classifier: MLPClassifier(activation='tanh', alpha=0.001,
              hidden_layer_sizes=(112, 112, 112), learning_rate_init=0.1,
              max_iter=3000, random_state=1, solver='sgd')
0.8680555555555556
	features: all features
	layers: 112
	test size: 0.4
	alpha: 0.1
	lambda: 1.0
	train accuracy: 0.9994212962962963
	test accuracy: 0.8680555555555556
	classifier: MLPClassifier(activation='tanh', alpha=1.0, hidden_layer_sizes=112,
              learning_rate_init=0.1, max_iter=3000, random_s

### User B

In [23]:
model_info_b_log = {} # key is just an int, values are info
automate_run(df2, model_info_b_log)



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=1.0
lambda=1000



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=1.0
lambda=100



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=1.0
lambda=1.0



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=1.0
lambda=0.1



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=1.0
lambda=0.01



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=1.0
lambda=0.001



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=1.0
lambda=0.0001



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=1.0
lambda=1e-05



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=0.1
lambda=1000



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=0.1
lambda=100



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=0.1
lambda=1.0



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=0.1
lambda=0.1



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=0.1
l

* * *
training accuracy: 100.0%
testing accuracy: 86.45833333333334%

Confusion Matrix: 
[[346  32  28]
 [ 22 314  22]
 [ 25  27 336]]

first 10 test values and predictions:
2 [0.59326601 0.00398405 0.40274993]
1 [0.19864151 0.79540979 0.0059487 ]
0 [9.96199826e-01 2.96602887e-03 8.34144723e-04]
1 [4.25044106e-05 9.99955467e-01 2.02888546e-06]
0 [9.99902994e-01 1.70572294e-08 9.69884567e-05]
2 [1.44017390e-04 1.08653910e-01 8.91202073e-01]
1 [7.25335972e-05 9.99926813e-01 6.53318728e-07]
2 [1.20235631e-07 4.50676157e-03 9.95493118e-01]
2 [9.60083565e-11 1.22045608e-06 9.99998779e-01]
1 [2.52581336e-07 5.46523863e-03 9.94534509e-01]


all features:
layers=112
test_size=0.4
alpha=1.0
lambda=0.001

* * *
training accuracy: 100.0%
testing accuracy: 84.72222222222221%

Confusion Matrix: 
[[357  30  18]
 [ 24 307  35]
 [ 34  35 312]]

first 10 test values and predictions:
0 [9.99959639e-01 2.07007645e-07 4.01538629e-05]
2 [3.90123828e-04 3.21450951e-03 9.96395367e-01]
1 [8.24097790e-06 3.243

* * *
training accuracy: 100.0%
testing accuracy: 85.85069444444444%

Confusion Matrix: 
[[337  24  19]
 [ 31 327  35]
 [ 17  37 325]]

first 10 test values and predictions:
2 [0.16346644 0.03112461 0.80540895]
0 [0.97830869 0.01185572 0.00983559]
0 [0.99362974 0.00445821 0.00191205]
2 [2.18428556e-02 5.04988825e-04 9.77652156e-01]
2 [0.00391277 0.35959323 0.636494  ]
1 [0.00184928 0.99715219 0.00099853]
2 [0.5316432  0.35334646 0.11501034]
1 [9.05131531e-06 9.94308975e-01 5.68197372e-03]
0 [9.97621147e-01 3.31238735e-04 2.04761395e-03]
1 [0.51764401 0.12642718 0.35592881]


all features:
layers=112
test_size=0.4
alpha=0.01
lambda=0.0001

* * *
training accuracy: 100.0%
testing accuracy: 84.02777777777779%

Confusion Matrix: 
[[315  35  33]
 [ 20 325  32]
 [ 22  42 328]]

first 10 test values and predictions:
2 [0.16355084 0.01225776 0.8241914 ]
1 [6.68888375e-03 9.93197959e-01 1.13157105e-04]
0 [0.99223142 0.00384337 0.00392521]
0 [0.95902833 0.02330681 0.01766486]
1 [0.00144098 0.995

C:\Users\Elijah\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:684: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (3000) reached and the optimization hasn't converged yet.
  warnings.warn(




all features:
layers=112
test_size=0.4
alpha=0.001
lambda=0.01



...



mean features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=0.01
lambda=1e-05



mean features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=0.001
lambda=1000



mean features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=0.001
lambda=100



mean features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=0.001
lambda=1.0



mean features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=0.001
lambda=0.1



mean features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=0.001
lambda=0.01



mean features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=0.001
lambda=0.001



mean features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=0.001
lambda=0.0001



mean features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=0.001
lambda=1e-05



mean features:
layers=(112, 56, 30, 10)
test_size=0.3
alpha=1.0
lambda=1000



mean features:
layers=(112, 56, 30, 10)
test_size=0.3
alpha=1.0
lambda=100



mean features:
layers=(112, 56, 30, 10)
test_size=0.3
alpha=1.0
lambda=1.0



mean features:
layers=(112, 56, 3



mean features:
layers=112
test_size=0.4
alpha=0.1
lambda=100



mean features:
layers=112
test_size=0.4
alpha=0.1
lambda=1.0



mean features:
layers=112
test_size=0.4
alpha=0.1
lambda=0.1

* * *
training accuracy: 100.0%
testing accuracy: 84.98263888888889%

Confusion Matrix: 
[[360  21  19]
 [ 37 306  36]
 [ 28  32 313]]

first 10 test values and predictions:
1 [0.87321601 0.12406649 0.0027175 ]
0 [0.84301697 0.1268037  0.03017933]
0 [0.9062477  0.04279392 0.05095838]
0 [0.97006805 0.02520702 0.00472493]
1 [0.17891672 0.40813223 0.41295104]
1 [0.77966145 0.20772992 0.01260863]
0 [0.9845243  0.00123822 0.01423749]
2 [0.05679047 0.25352933 0.6896802 ]
0 [0.74853441 0.24638228 0.00508331]
1 [0.12362865 0.87357003 0.00280132]


mean features:
layers=112
test_size=0.4
alpha=0.1
lambda=0.01

* * *
training accuracy: 100.0%
testing accuracy: 85.24305555555556%

Confusion Matrix: 
[[327  34  17]
 [ 29 329  41]
 [ 16  33 326]]

first 10 test values and predictions:
2 [0.08442385 0.37931851 



mean features:
layers=112
test_size=0.3
alpha=1.0
lambda=1000



mean features:
layers=112
test_size=0.3
alpha=1.0
lambda=100



mean features:
layers=112
test_size=0.3
alpha=1.0
lambda=1.0



mean features:
layers=112
test_size=0.3
alpha=1.0
lambda=0.1

* * *
training accuracy: 95.68452380952381%
testing accuracy: 81.82870370370371%

Confusion Matrix: 
[[254  13  13]
 [ 36 239  32]
 [ 23  40 214]]

first 10 test values and predictions:
1 [0.0169847  0.98185407 0.00116124]
2 [0.69226436 0.0381633  0.26957234]
1 [0.23112621 0.13256616 0.63630764]
1 [0.09185648 0.90256314 0.00558037]
0 [9.89040435e-01 7.32826505e-05 1.08862825e-02]
0 [0.96026204 0.02929388 0.01044408]
0 [0.81805759 0.08245516 0.09948725]
2 [3.93602306e-06 1.63423259e-05 9.99979722e-01]
0 [0.98105755 0.00270412 0.01623833]
2 [7.38735528e-04 1.30818558e-01 8.68442706e-01]


mean features:
layers=112
test_size=0.3
alpha=1.0
lambda=0.01

* * *
training accuracy: 100.0%
testing accuracy: 84.72222222222221%

Confusion Matrix

* * *
training accuracy: 100.0%
testing accuracy: 82.87037037037037%

Confusion Matrix: 
[[246  25  23]
 [ 20 228  27]
 [ 21  32 242]]

first 10 test values and predictions:
2 [2.28730119e-04 5.48857583e-01 4.50913687e-01]
0 [9.98477829e-01 1.21783980e-03 3.04331198e-04]
2 [5.11526595e-10 8.68588560e-09 9.99999991e-01]
1 [0.00340511 0.9920346  0.00456028]
1 [3.88233775e-04 9.90708221e-01 8.90354564e-03]
2 [2.41311381e-06 5.50953232e-04 9.99446634e-01]
2 [0.78366846 0.00214791 0.21418363]
1 [0.00626853 0.98314278 0.01058868]
0 [0.48072572 0.40618758 0.11308671]
0 [0.80001661 0.0098698  0.19011359]


mean features:
layers=112
test_size=0.3
alpha=0.01
lambda=0.001

* * *
training accuracy: 100.0%
testing accuracy: 85.06944444444444%

Confusion Matrix: 
[[248  17  13]
 [ 30 246  25]
 [ 16  28 241]]

first 10 test values and predictions:
1 [0.40686513 0.58959692 0.00353794]
1 [1.86890685e-07 9.99954028e-01 4.57847599e-05]
0 [0.99710214 0.00151957 0.00137829]
2 [0.00273693 0.02506375 0.97219



mean features:
layers=(112, 112, 112)
test_size=0.4
alpha=0.01
lambda=100



mean features:
layers=(112, 112, 112)
test_size=0.4
alpha=0.01
lambda=1.0



mean features:
layers=(112, 112, 112)
test_size=0.4
alpha=0.01
lambda=0.1



mean features:
layers=(112, 112, 112)
test_size=0.4
alpha=0.01
lambda=0.01



mean features:
layers=(112, 112, 112)
test_size=0.4
alpha=0.01
lambda=0.001



mean features:
layers=(112, 112, 112)
test_size=0.4
alpha=0.01
lambda=0.0001



mean features:
layers=(112, 112, 112)
test_size=0.4
alpha=0.01
lambda=1e-05



mean features:
layers=(112, 112, 112)
test_size=0.4
alpha=0.001
lambda=1000



mean features:
layers=(112, 112, 112)
test_size=0.4
alpha=0.001
lambda=100



mean features:
layers=(112, 112, 112)
test_size=0.4
alpha=0.001
lambda=1.0



mean features:
layers=(112, 112, 112)
test_size=0.4
alpha=0.001
lambda=0.1



mean features:
layers=(112, 112, 112)
test_size=0.4
alpha=0.001
lambda=0.01



mean features:
layers=(112, 112, 112)
test_size=0.4
alpha=0

In [24]:
# looking at the top models
show_above(model_info_b_log, 0.88)

0.9155092592592593
	features: all features
	layers: 112
	test size: 0.3
	alpha: 0.01
	lambda: 0.001
	train accuracy: 1.0
	test accuracy: 0.9155092592592593
	classifier: MLPClassifier(activation='logistic', alpha=0.001, hidden_layer_sizes=112,
              learning_rate_init=0.01, max_iter=3000, random_state=1,
              solver='sgd')
0.9050925925925926
	features: all features
	layers: 112
	test size: 0.3
	alpha: 0.1
	lambda: 1e-05
	train accuracy: 1.0
	test accuracy: 0.9050925925925926
	classifier: MLPClassifier(activation='logistic', alpha=1e-05, hidden_layer_sizes=112,
              learning_rate_init=0.1, max_iter=3000, random_state=1,
              solver='sgd')
0.9004629629629629
	features: all features
	layers: 112
	test size: 0.3
	alpha: 1.0
	lambda: 0.01
	train accuracy: 1.0
	test accuracy: 0.9004629629629629
	classifier: MLPClassifier(activation='logistic', alpha=0.01, hidden_layer_sizes=112,
              learning_rate_init=1.0, max_iter=3000, random_state=1,
           

In [25]:
model_info_b_tanh = {} # key is just an int, values are info
automate_run(df2, model_info_b_tanh, act='tanh')



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=1.0
lambda=1000



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=1.0
lambda=100



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=1.0
lambda=1.0



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=1.0
lambda=0.1



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=1.0
lambda=0.01



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=1.0
lambda=0.001



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=1.0
lambda=0.0001



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=1.0
lambda=1e-05



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=0.1
lambda=1000



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=0.1
lambda=100



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=0.1
lambda=1.0

* * *
training accuracy: 100.0%
testing accuracy: 90.27777777777779%

Confusion Matrix: 
[[373  18  16]
 [ 17 340  24]
 [ 13  24 327]]

first 1



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=0.01
lambda=1e-05

* * *
training accuracy: 100.0%
testing accuracy: 81.68402777777779%

Confusion Matrix: 
[[331  22  19]
 [ 34 311  49]
 [ 40  47 299]]

first 10 test values and predictions:
0 [9.96630529e-01 2.42212373e-03 9.47347735e-04]
2 [4.73553351e-05 1.36871394e-04 9.99815773e-01]
1 [4.36740216e-04 9.97025418e-01 2.53784206e-03]
2 [0.00392979 0.02666674 0.96940346]
0 [9.92699156e-01 7.12264055e-03 1.78203423e-04]
1 [7.63422453e-05 9.97487014e-01 2.43664372e-03]
2 [0.07376628 0.00201488 0.92421884]
2 [6.77134410e-05 2.94066664e-03 9.96991620e-01]
2 [0.00332735 0.00407399 0.99259866]
1 [0.98936194 0.00882934 0.00180872]


all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=0.001
lambda=1000



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=0.001
lambda=100



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=0.001
lambda=1.0

* * *
training accuracy: 100.0%
testing accuracy: 85.5902777777777

* * *
training accuracy: 100.0%
testing accuracy: 85.30092592592592%

Confusion Matrix: 
[[246  16  22]
 [ 21 233  26]
 [ 20  22 258]]

first 10 test values and predictions:
2 [4.27455423e-05 2.73706107e-04 9.99683548e-01]
1 [3.04394988e-02 9.69500775e-01 5.97263078e-05]
2 [5.02131895e-06 5.41416295e-02 9.45853349e-01]
0 [2.03138188e-04 9.77870042e-01 2.19268202e-02]
2 [5.74348079e-05 6.75670685e-05 9.99874998e-01]
0 [1.54801325e-02 9.84228536e-01 2.91331804e-04]
0 [0.70538667 0.29215924 0.00245409]
1 [3.86869634e-05 9.99769630e-01 1.91682696e-04]
2 [8.31137688e-03 9.28832101e-04 9.90759791e-01]
2 [3.46438129e-04 1.90224221e-04 9.99463338e-01]


all features:
layers=(112, 56, 30, 10)
test_size=0.3
alpha=0.1
lambda=1e-05

* * *
training accuracy: 100.0%
testing accuracy: 86.57407407407408%

Confusion Matrix: 
[[263  21  14]
 [ 21 244  24]
 [  7  29 241]]

first 10 test values and predictions:
1 [5.76582374e-05 9.99692502e-01 2.49840044e-04]
2 [6.89348069e-05 1.99472412e-03 9.97936341e-0

* * *
training accuracy: 100.0%
testing accuracy: 82.87037037037037%

Confusion Matrix: 
[[247  21  19]
 [ 20 246  28]
 [ 22  38 223]]

first 10 test values and predictions:
0 [9.85947739e-01 1.35521767e-04 1.39167389e-02]
1 [7.25491225e-04 9.94440618e-01 4.83389064e-03]
2 [8.03911145e-05 8.55913576e-04 9.99063695e-01]
1 [1.49582341e-03 9.97563794e-01 9.40382608e-04]
0 [9.48531271e-01 5.11228251e-02 3.45904078e-04]
1 [0.00291061 0.15572591 0.84136348]
0 [9.98848616e-01 2.56553301e-04 8.94830538e-04]
2 [5.36490447e-04 8.35272138e-04 9.98628237e-01]
0 [0.08683396 0.89561857 0.01754748]
1 [0.00236951 0.99618147 0.00144902]


all features:
layers=(112, 56, 30, 10)
test_size=0.3
alpha=0.001
lambda=0.0001

* * *
training accuracy: 100.0%
testing accuracy: 82.4074074074074%

Confusion Matrix: 
[[236  22  18]
 [ 21 229  35]
 [ 21  35 247]]

first 10 test values and predictions:
2 [0.00154475 0.16028724 0.83816801]
1 [0.01889526 0.05651986 0.92458488]
0 [9.91575979e-01 8.00792051e-03 4.16100614

* * *
training accuracy: 100.0%
testing accuracy: 86.37152777777779%

Confusion Matrix: 
[[325  20  20]
 [ 25 324  35]
 [ 20  37 346]]

first 10 test values and predictions:
0 [3.21695582e-02 9.67755178e-01 7.52634775e-05]
0 [9.99996743e-01 3.20180146e-06 5.56385725e-08]
2 [1.67502141e-04 9.82908309e-04 9.98849590e-01]
1 [6.62330253e-10 9.99995985e-01 4.01471501e-06]
2 [3.83827032e-05 5.11399422e-02 9.48821675e-01]
0 [9.98211291e-01 1.03922953e-03 7.49479796e-04]
0 [9.99934670e-01 3.21238372e-06 6.21172701e-05]
2 [0.26209012 0.01443267 0.72347721]
1 [1.10044083e-04 1.02750463e-03 9.98862451e-01]
2 [0.00349328 0.00655541 0.98995131]


all features:
layers=112
test_size=0.4
alpha=0.1
lambda=1e-05

* * *
training accuracy: 100.0%
testing accuracy: 82.29166666666666%

Confusion Matrix: 
[[333  35  28]
 [ 36 294  38]
 [ 23  44 321]]

first 10 test values and predictions:
0 [9.90445134e-01 9.55484939e-03 1.62464876e-08]
1 [2.08855996e-04 9.47871433e-01 5.19197111e-02]
0 [9.93554616e-01 6.007

...



mean features:
layers=112
test_size=0.3
alpha=0.01
lambda=1.0

* * *
training accuracy: 94.29563492063492%
testing accuracy: 84.49074074074075%

Confusion Matrix: 
[[261  22  15]
 [ 24 238  19]
 [ 13  41 231]]

first 10 test values and predictions:
2 [0.03056015 0.40645611 0.56298374]
2 [0.10188771 0.24455678 0.65355552]
2 [0.03706002 0.01783718 0.94510279]
0 [0.86843863 0.02712381 0.10443756]
0 [0.91675382 0.06997431 0.01327187]
2 [0.0379068  0.39350035 0.56859285]
2 [0.42288607 0.08426243 0.4928515 ]
0 [0.36181434 0.63230822 0.00587744]
2 [0.00678161 0.02787171 0.96534668]
2 [0.0786506  0.18170163 0.73964777]


mean features:
layers=112
test_size=0.3
alpha=0.01
lambda=0.1

* * *
training accuracy: 100.0%
testing accuracy: 88.19444444444444%

Confusion Matrix: 
[[258  19  17]
 [ 12 239  15]
 [ 14  25 265]]

first 10 test values and predictions:
0 [0.95334583 0.04008108 0.00657309]
2 [2.96452322e-04 2.60923307e-02 9.73611217e-01]
1 [1.59608863e-03 9.98159858e-01 2.44053191e-04]
0 [9.



mean features:
layers=(112, 112, 112)
test_size=0.4
alpha=1.0
lambda=100



mean features:
layers=(112, 112, 112)
test_size=0.4
alpha=1.0
lambda=1.0



mean features:
layers=(112, 112, 112)
test_size=0.4
alpha=1.0
lambda=0.1



mean features:
layers=(112, 112, 112)
test_size=0.4
alpha=1.0
lambda=0.01



mean features:
layers=(112, 112, 112)
test_size=0.4
alpha=1.0
lambda=0.001



mean features:
layers=(112, 112, 112)
test_size=0.4
alpha=1.0
lambda=0.0001



mean features:
layers=(112, 112, 112)
test_size=0.4
alpha=1.0
lambda=1e-05



mean features:
layers=(112, 112, 112)
test_size=0.4
alpha=0.1
lambda=1000



mean features:
layers=(112, 112, 112)
test_size=0.4
alpha=0.1
lambda=100



mean features:
layers=(112, 112, 112)
test_size=0.4
alpha=0.1
lambda=1.0

* * *
training accuracy: 100.0%
testing accuracy: 86.89236111111111%

Confusion Matrix: 
[[338  23  32]
 [ 20 321  20]
 [ 18  38 342]]

first 10 test values and predictions:
0 [9.98925939e-01 1.01909328e-03 5.49672708e-05]
1 [1.527

* * *
training accuracy: 100.0%
testing accuracy: 85.41666666666666%

Confusion Matrix: 
[[315  26  23]
 [ 28 335  36]
 [ 34  21 334]]

first 10 test values and predictions:
0 [9.99997946e-01 2.62932703e-11 2.05382148e-06]
0 [9.99995088e-01 1.99478324e-08 4.89216205e-06]
2 [3.52107152e-08 3.32174557e-06 9.99996643e-01]
0 [9.68533640e-01 3.14657038e-02 6.56205005e-07]
1 [0.41592276 0.56508113 0.01899611]
0 [9.99960317e-01 3.88366577e-05 8.46363768e-07]
0 [9.99692811e-01 3.06226983e-04 9.62378678e-07]
1 [0.03949471 0.95925607 0.00124922]
2 [4.55879920e-04 4.78287594e-06 9.99539337e-01]
2 [5.15063003e-04 1.81739052e-03 9.97667546e-01]


mean features:
layers=(112, 112, 112)
test_size=0.4
alpha=0.01
lambda=1e-05

* * *
training accuracy: 100.0%
testing accuracy: 83.68055555555556%

Confusion Matrix: 
[[326  40  33]
 [ 21 312  36]
 [ 20  38 326]]

first 10 test values and predictions:
1 [0.00052947 0.50065436 0.49881617]
1 [1.48822520e-07 9.93627640e-01 6.37221078e-03]
0 [0.88300626 0.06500

* * *
training accuracy: 100.0%
testing accuracy: 87.84722222222221%

Confusion Matrix: 
[[262  14  11]
 [ 16 254  26]
 [ 17  21 243]]

first 10 test values and predictions:
2 [1.26407526e-08 1.31826676e-04 9.99868161e-01]
2 [7.50821647e-05 5.23417942e-06 9.99919684e-01]
1 [1.31836304e-04 9.99866549e-01 1.61502521e-06]
2 [2.15050713e-04 1.58298163e-02 9.83955133e-01]
2 [4.57256460e-02 1.82184158e-05 9.54256136e-01]
2 [1.44476266e-04 3.98284090e-01 6.01571434e-01]
0 [2.03379101e-02 1.85730834e-06 9.79660233e-01]
1 [1.03617950e-03 9.98963820e-01 7.83845508e-10]
0 [9.98581999e-01 1.33690098e-03 8.10996989e-05]
1 [1.82639425e-10 9.99999805e-01 1.94831078e-07]


mean features:
layers=(112, 112, 112)
test_size=0.3
alpha=0.1
lambda=1e-05

* * *
training accuracy: 100.0%
testing accuracy: 86.92129629629629%

Confusion Matrix: 
[[261  21  16]
 [ 18 248  18]
 [ 13  27 242]]

first 10 test values and predictions:
0 [9.99999071e-01 9.00361565e-07 2.88462475e-08]
1 [3.13684086e-04 9.98836323e-01 8.

* * *
training accuracy: 100.0%
testing accuracy: 85.41666666666666%

Confusion Matrix: 
[[246  24  19]
 [ 17 229  22]
 [ 20  24 263]]

first 10 test values and predictions:
0 [9.93862705e-01 6.13161267e-03 5.68260603e-06]
0 [9.99791475e-01 2.06397859e-04 2.12700318e-06]
1 [1.20012191e-05 9.57546257e-01 4.24417417e-02]
0 [0.49518267 0.49794501 0.00687233]
2 [1.16443948e-03 5.98921481e-07 9.98834962e-01]
0 [0.72929978 0.01142651 0.2592737 ]
2 [2.40527192e-04 6.07852683e-02 9.38974205e-01]
2 [6.14900818e-09 8.42220750e-07 9.99999152e-01]
2 [1.89274386e-04 2.26529658e-01 7.73281068e-01]
1 [2.35238024e-05 9.95500444e-01 4.47603250e-03]


mean features:
layers=(112, 112, 112)
test_size=0.3
alpha=0.001
lambda=0.0001

* * *
training accuracy: 100.0%
testing accuracy: 83.33333333333334%

Confusion Matrix: 
[[255  27  18]
 [ 21 227  28]
 [ 25  25 238]]

first 10 test values and predictions:
0 [7.24275488e-01 2.75724212e-01 3.00065461e-07]
2 [4.40976172e-06 2.40330474e-10 9.99995590e-01]
0 [4.54

In [26]:
# looking at the top models
show_above(model_info_b_tanh, 0.88)

0.9224537037037037
	features: all features
	layers: 112
	test size: 0.3
	alpha: 0.1
	lambda: 0.1
	train accuracy: 1.0
	test accuracy: 0.9224537037037037
	classifier: MLPClassifier(activation='tanh', alpha=0.1, hidden_layer_sizes=112,
              learning_rate_init=0.1, max_iter=3000, random_state=1,
              solver='sgd')
0.9166666666666666
	features: all features
	layers: 112
	test size: 0.3
	alpha: 0.01
	lambda: 1.0
	train accuracy: 0.9950396825396826
	test accuracy: 0.9166666666666666
	classifier: MLPClassifier(activation='tanh', alpha=1.0, hidden_layer_sizes=112,
              learning_rate_init=0.01, max_iter=3000, random_state=1,
              solver='sgd')
0.9140625
	features: all features
	layers: 112
	test size: 0.4
	alpha: 1.0
	lambda: 0.1
	train accuracy: 1.0
	test accuracy: 0.9140625
	classifier: MLPClassifier(activation='tanh', alpha=0.1, hidden_layer_sizes=112,
              learning_rate_init=1.0, max_iter=3000, random_state=1,
              solver='sgd')
0.908854

### User C

In [27]:
model_info_c_log = {} # key is just an int, values are info 
automate_run(df2, model_info_c_log)



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=1.0
lambda=1000



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=1.0
lambda=100



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=1.0
lambda=1.0



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=1.0
lambda=0.1



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=1.0
lambda=0.01



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=1.0
lambda=0.001



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=1.0
lambda=0.0001



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=1.0
lambda=1e-05



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=0.1
lambda=1000



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=0.1
lambda=100



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=0.1
lambda=1.0



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=0.1
lambda=0.1



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=0.1
l

* * *
training accuracy: 100.0%
testing accuracy: 86.63194444444444%

Confusion Matrix: 
[[336  24  27]
 [ 19 342  31]
 [ 14  39 320]]

first 10 test values and predictions:
2 [8.13642688e-04 8.52654396e-05 9.99101092e-01]
0 [2.16862015e-02 2.61200244e-04 9.78052598e-01]
2 [1.07112499e-04 3.33572507e-04 9.99559315e-01]
2 [1.11740828e-04 9.34079978e-04 9.98954179e-01]
1 [0.31894757 0.39640852 0.28464391]
2 [1.45398340e-04 6.19015553e-03 9.93664446e-01]
1 [3.83208370e-09 9.99998589e-01 1.40685132e-06]
1 [9.83469430e-05 9.99839188e-01 6.24649045e-05]
0 [9.99408366e-01 5.29358940e-06 5.86340171e-04]
2 [1.09924495e-02 2.91457186e-05 9.88978405e-01]


all features:
layers=112
test_size=0.4
alpha=1.0
lambda=0.001

* * *
training accuracy: 100.0%
testing accuracy: 84.98263888888889%

Confusion Matrix: 
[[314  28  27]
 [ 27 336  24]
 [ 30  37 329]]

first 10 test values and predictions:
0 [9.99886095e-01 6.11100746e-08 1.13844075e-04]
1 [0.96240154 0.0082618  0.02933666]
1 [1.15548607e-05 9.995

* * *
training accuracy: 100.0%
testing accuracy: 86.02430555555556%

Confusion Matrix: 
[[332  28  26]
 [ 20 343  25]
 [ 20  42 316]]

first 10 test values and predictions:
1 [0.24131445 0.73009145 0.02859409]
0 [0.9527759 0.0300488 0.0171753]
1 [0.04985746 0.78622092 0.16392161]
0 [0.05456634 0.92839095 0.01704271]
1 [0.00622036 0.99003429 0.00374535]
2 [0.00109083 0.00602945 0.99287973]
0 [0.99615913 0.00213763 0.00170324]
2 [0.09208829 0.67339245 0.23451926]
0 [8.31876838e-01 5.90691462e-04 1.67532471e-01]
0 [0.97621236 0.0222398  0.00154784]


all features:
layers=112
test_size=0.4
alpha=0.01
lambda=0.01

* * *
training accuracy: 100.0%
testing accuracy: 86.97916666666666%

Confusion Matrix: 
[[323  21  17]
 [ 20 341  37]
 [ 14  41 338]]

first 10 test values and predictions:
1 [3.46846942e-04 9.97103226e-01 2.54992676e-03]
1 [2.27930549e-04 9.90611587e-01 9.16048211e-03]
2 [0.00380879 0.49508023 0.50111098]
2 [0.13884302 0.01479525 0.84636173]
1 [0.01805069 0.85772735 0.12422196]

C:\Users\Elijah\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:684: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (3000) reached and the optimization hasn't converged yet.
  warnings.warn(




all features:
layers=112
test_size=0.4
alpha=0.001
lambda=0.01



...

* * *
training accuracy: 100.0%
testing accuracy: 85.9375%

Confusion Matrix: 
[[342  32  24]
 [ 25 312  32]
 [ 17  32 336]]

first 10 test values and predictions:
1 [1.63974397e-04 9.81375762e-01 1.84602634e-02]
0 [0.60260362 0.0682721  0.32912429]
2 [2.03883541e-07 1.53137953e-05 9.99984482e-01]
2 [2.70838949e-04 1.41693716e-04 9.99587467e-01]
1 [0.03987979 0.93541866 0.02470155]
1 [5.05389763e-04 9.81482211e-01 1.80123992e-02]
0 [9.96804651e-01 2.50173009e-05 3.17033217e-03]
0 [7.80834170e-01 2.19113648e-01 5.21818588e-05]
1 [6.45171548e-04 9.99354480e-01 3.48730205e-07]
2 [2.08078370e-04 6.85938007e-03 9.92932542e-01]


mean features:
layers=112
test_size=0.4
alpha=1.0
lambda=0.001

* * *
training accuracy: 100.0%
testing accuracy: 82.37847222222221%

Confusion Matrix: 
[[313  43  25]
 [ 29 322  23]
 [ 29  54 314]]

first 10 test values and predictions:
2 [7.07215641e-04 1.07819168e-01 8.91473617e-01]
0 [9.99778465e-01 1.79369829e-04 4.21648216e-05]
1 [0.18594892 0.80139114 0.01265

* * *
training accuracy: 100.0%
testing accuracy: 83.33333333333334%

Confusion Matrix: 
[[318  37  28]
 [ 25 309  32]
 [ 19  51 333]]

first 10 test values and predictions:
0 [0.99368723 0.00362245 0.00269032]
2 [0.03381506 0.00486301 0.96132193]
2 [0.11565026 0.01582924 0.86852051]
1 [0.32792423 0.56306762 0.10900815]
0 [0.96802922 0.01484543 0.01712536]
2 [0.01716606 0.00526146 0.97757248]
0 [9.99894458e-01 6.90097926e-05 3.65318239e-05]
2 [0.21495032 0.07193417 0.7131155 ]
2 [0.02732457 0.80042437 0.17225107]
2 [2.20247816e-12 2.42177709e-04 9.99757822e-01]


mean features:
layers=112
test_size=0.4
alpha=0.01
lambda=0.0001

* * *
training accuracy: 99.94212962962963%
testing accuracy: 84.375%

Confusion Matrix: 
[[334  32  24]
 [ 35 297  36]
 [ 13  40 341]]

first 10 test values and predictions:
1 [0.01192754 0.9844536  0.00361886]
2 [5.24601919e-03 8.69656931e-05 9.94667015e-01]
1 [0.02174355 0.42341621 0.55484024]
2 [1.90471439e-04 1.15938775e-02 9.88215651e-01]
1 [0.00279016 0.9

* * *
training accuracy: 100.0%
testing accuracy: 84.95370370370371%

Confusion Matrix: 
[[259  15  15]
 [ 29 224  21]
 [ 16  34 251]]

first 10 test values and predictions:
2 [2.16487605e-10 3.54709066e-06 9.99996453e-01]
2 [5.53903568e-05 6.98302382e-03 9.92961586e-01]
2 [1.04098997e-03 5.02762787e-04 9.98456247e-01]
2 [0.46340631 0.00405344 0.53254025]
2 [1.31683240e-08 1.04293864e-07 9.99999883e-01]
1 [9.90566060e-01 9.08403997e-03 3.49899893e-04]
1 [0.0234665  0.88847413 0.08805937]
2 [0.30905239 0.11526672 0.57568089]
1 [7.82200187e-04 9.14251216e-01 8.49665834e-02]
2 [2.11156479e-16 4.12807788e-10 1.00000000e+00]


mean features:
layers=112
test_size=0.3
alpha=0.1
lambda=0.0001

* * *
training accuracy: 100.0%
testing accuracy: 87.61574074074075%

Confusion Matrix: 
[[253  21  16]
 [ 17 226  21]
 [ 10  22 278]]

first 10 test values and predictions:
1 [0.11328813 0.67882551 0.20788636]
2 [0.00747027 0.03963167 0.95289806]
2 [2.73423927e-05 4.18394765e-04 9.99554263e-01]
2 [6.569



mean features:
layers=(112, 112, 112)
test_size=0.4
alpha=0.1
lambda=100



mean features:
layers=(112, 112, 112)
test_size=0.4
alpha=0.1
lambda=1.0



mean features:
layers=(112, 112, 112)
test_size=0.4
alpha=0.1
lambda=0.1

* * *
training accuracy: 100.0%
testing accuracy: 85.41666666666666%

Confusion Matrix: 
[[337  27  32]
 [ 22 315  38]
 [ 10  39 332]]

first 10 test values and predictions:
1 [3.38050363e-02 9.66033064e-01 1.61899308e-04]
0 [0.99760622 0.00109473 0.00129905]
0 [9.70827347e-01 8.99537777e-05 2.90826992e-02]
0 [9.99924762e-01 7.52326309e-05 5.31775821e-09]
2 [6.47775445e-03 1.29757248e-04 9.93392488e-01]
0 [9.99988753e-01 1.06247756e-05 6.21841046e-07]
2 [1.57450557e-04 1.65034878e-05 9.99826046e-01]
2 [0.00292438 0.02635461 0.97072102]
1 [0.00103137 0.84376015 0.15520848]
1 [0.03477312 0.73663013 0.22859675]


mean features:
layers=(112, 112, 112)
test_size=0.4
alpha=0.1
lambda=0.01

* * *
training accuracy: 100.0%
testing accuracy: 82.37847222222221%

Confusion



mean features:
layers=(112, 112, 112)
test_size=0.3
alpha=0.01
lambda=100



mean features:
layers=(112, 112, 112)
test_size=0.3
alpha=0.01
lambda=1.0



mean features:
layers=(112, 112, 112)
test_size=0.3
alpha=0.01
lambda=0.1



mean features:
layers=(112, 112, 112)
test_size=0.3
alpha=0.01
lambda=0.01



mean features:
layers=(112, 112, 112)
test_size=0.3
alpha=0.01
lambda=0.001



mean features:
layers=(112, 112, 112)
test_size=0.3
alpha=0.01
lambda=0.0001



mean features:
layers=(112, 112, 112)
test_size=0.3
alpha=0.01
lambda=1e-05



mean features:
layers=(112, 112, 112)
test_size=0.3
alpha=0.001
lambda=1000



mean features:
layers=(112, 112, 112)
test_size=0.3
alpha=0.001
lambda=100



mean features:
layers=(112, 112, 112)
test_size=0.3
alpha=0.001
lambda=1.0



mean features:
layers=(112, 112, 112)
test_size=0.3
alpha=0.001
lambda=0.1



mean features:
layers=(112, 112, 112)
test_size=0.3
alpha=0.001
lambda=0.01



mean features:
layers=(112, 112, 112)
test_size=0.3
alpha=0

In [28]:
# looking at the top models
show_above(model_info_c_log, 0.88)

0.9189814814814815
	features: all features
	layers: 112
	test size: 0.3
	alpha: 0.1
	lambda: 0.1
	train accuracy: 1.0
	test accuracy: 0.9189814814814815
	classifier: MLPClassifier(activation='logistic', alpha=0.1, hidden_layer_sizes=112,
              learning_rate_init=0.1, max_iter=3000, random_state=1,
              solver='sgd')
0.9039351851851852
	features: all features
	layers: 112
	test size: 0.3
	alpha: 0.1
	lambda: 1e-05
	train accuracy: 1.0
	test accuracy: 0.9039351851851852
	classifier: MLPClassifier(activation='logistic', alpha=1e-05, hidden_layer_sizes=112,
              learning_rate_init=0.1, max_iter=3000, random_state=1,
              solver='sgd')
0.8993055555555556
	features: all features
	layers: 112
	test size: 0.3
	alpha: 0.01
	lambda: 0.01
	train accuracy: 1.0
	test accuracy: 0.8993055555555556
	classifier: MLPClassifier(activation='logistic', alpha=0.01, hidden_layer_sizes=112,
              learning_rate_init=0.01, max_iter=3000, random_state=1,
              s

In [29]:
model_info_c_tanh = {} # key is just an int, values are info
automate_run(df2, model_info_c_tanh, act='tanh')



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=1.0
lambda=1000



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=1.0
lambda=100



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=1.0
lambda=1.0



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=1.0
lambda=0.1



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=1.0
lambda=0.01



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=1.0
lambda=0.001



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=1.0
lambda=0.0001



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=1.0
lambda=1e-05



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=0.1
lambda=1000



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=0.1
lambda=100



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=0.1
lambda=1.0

* * *
training accuracy: 100.0%
testing accuracy: 87.15277777777779%

Confusion Matrix: 
[[335  21  17]
 [ 20 348  31]
 [ 25  34 321]]

first 1

* * *
training accuracy: 100.0%
testing accuracy: 81.85763888888889%

Confusion Matrix: 
[[318  34  34]
 [ 27 317  48]
 [ 30  36 308]]

first 10 test values and predictions:
1 [2.04374416e-01 7.94934064e-01 6.91519903e-04]
2 [4.66289917e-04 6.44850640e-04 9.98888859e-01]
0 [9.98181072e-01 2.50025385e-05 1.79392581e-03]
0 [9.99148340e-01 2.81443322e-04 5.70216478e-04]
0 [0.36659582 0.01006977 0.62333441]
2 [2.40587375e-03 2.37903363e-04 9.97356223e-01]
0 [7.32184787e-01 5.51553196e-04 2.67263660e-01]
1 [0.15765027 0.66578353 0.17656621]
2 [0.01301828 0.23881166 0.74817006]
1 [1.77972061e-04 6.41699353e-01 3.58122674e-01]


all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=0.01
lambda=1e-05

* * *
training accuracy: 100.0%
testing accuracy: 81.68402777777779%

Confusion Matrix: 
[[333  27  26]
 [ 32 316  46]
 [ 29  51 292]]

first 10 test values and predictions:
0 [9.94105047e-01 1.38327183e-04 5.75662603e-03]
1 [0.95177262 0.00116701 0.04706037]
2 [6.28618888e-04 2.96593699e-04

* * *
training accuracy: 100.0%
testing accuracy: 87.84722222222221%

Confusion Matrix: 
[[268  16  18]
 [ 10 256  23]
 [ 15  23 235]]

first 10 test values and predictions:
2 [7.32585525e-05 1.96086066e-05 9.99907133e-01]
2 [5.93302864e-07 3.90795228e-04 9.99608611e-01]
2 [3.18509854e-05 7.54606640e-01 2.45361509e-01]
0 [9.99790719e-01 2.11431780e-07 2.09069744e-04]
1 [1.48806688e-04 4.48833667e-01 5.51017526e-01]
2 [2.37732076e-07 1.97875162e-04 9.99801887e-01]
0 [9.95145031e-01 2.91167748e-04 4.56380124e-03]
1 [1.91091984e-06 9.99940103e-01 5.79863494e-05]
2 [7.11642334e-06 9.99098644e-01 8.94239917e-04]
1 [1.37740770e-04 9.99551136e-01 3.11123478e-04]


all features:
layers=(112, 56, 30, 10)
test_size=0.3
alpha=0.1
lambda=0.001

* * *
training accuracy: 100.0%
testing accuracy: 86.22685185185185%

Confusion Matrix: 
[[257  19  25]
 [ 18 243  16]
 [ 14  27 245]]

first 10 test values and predictions:
0 [9.99866923e-01 4.71350698e-05 8.59416172e-05]
0 [9.97519747e-01 2.42758712e-03 5

* * *
training accuracy: 100.0%
testing accuracy: 81.01851851851852%

Confusion Matrix: 
[[241  16  32]
 [ 27 226  39]
 [ 25  25 233]]

first 10 test values and predictions:
1 [0.13002163 0.00681749 0.86316087]
1 [0.00120706 0.99558551 0.00320743]
2 [0.03307277 0.0085187  0.95840854]
1 [0.00948624 0.98190915 0.0086046 ]
0 [0.60502714 0.03059605 0.36437681]
1 [1.02699150e-04 9.94596855e-01 5.30044633e-03]
2 [0.02336728 0.00184598 0.97478674]
2 [1.71410026e-04 2.41886023e-03 9.97409730e-01]
1 [2.50644181e-04 9.78171027e-01 2.15783286e-02]
1 [8.59029740e-04 9.92835324e-01 6.30564668e-03]


all features:
layers=(112, 56, 30, 10)
test_size=0.3
alpha=0.001
lambda=0.01

* * *
training accuracy: 100.0%
testing accuracy: 83.79629629629629%

Confusion Matrix: 
[[242  18  23]
 [ 16 244  29]
 [ 22  32 238]]

first 10 test values and predictions:
2 [8.54956936e-05 9.23530682e-03 9.90679197e-01]
2 [8.02838565e-04 7.59209411e-03 9.91605067e-01]
1 [0.00138472 0.99538704 0.00322824]
2 [8.63782685e-01 3

* * *
training accuracy: 100.0%
testing accuracy: 87.23958333333334%

Confusion Matrix: 
[[334  32  19]
 [ 14 347  32]
 [ 15  35 324]]

first 10 test values and predictions:
1 [0.00264953 0.94504889 0.05230158]
0 [5.03078954e-01 4.96732801e-01 1.88245131e-04]
0 [9.97452375e-01 5.53268704e-04 1.99435622e-03]
1 [0.00346917 0.98200744 0.01452339]
2 [0.03248383 0.05343924 0.91407692]
1 [1.88097844e-04 9.99280194e-01 5.31707671e-04]
1 [3.04043681e-03 9.96922511e-01 3.70522279e-05]
0 [9.86766884e-01 1.31053497e-02 1.27765947e-04]
2 [1.90763596e-07 2.88175919e-09 9.99999806e-01]
1 [2.69147541e-06 9.99957045e-01 4.02638441e-05]


all features:
layers=112
test_size=0.4
alpha=0.1
lambda=0.01

* * *
training accuracy: 100.0%
testing accuracy: 85.24305555555556%

Confusion Matrix: 
[[338  25  37]
 [ 28 320  39]
 [ 19  22 324]]

first 10 test values and predictions:
1 [0.39333407 0.45050232 0.1561636 ]
1 [0.04565377 0.9378384  0.01650783]
1 [2.74818794e-05 9.99308858e-01 6.63660146e-04]
2 [8.768846

...



mean features:
layers=112
test_size=0.3
alpha=1.0
lambda=0.0001



mean features:
layers=112
test_size=0.3
alpha=1.0
lambda=1e-05



mean features:
layers=112
test_size=0.3
alpha=0.1
lambda=1000



mean features:
layers=112
test_size=0.3
alpha=0.1
lambda=100



mean features:
layers=112
test_size=0.3
alpha=0.1
lambda=1.0

* * *
training accuracy: 92.1626984126984%
testing accuracy: 83.2175925925926%

Confusion Matrix: 
[[237  27  25]
 [ 10 222  49]
 [ 12  22 260]]

first 10 test values and predictions:
2 [0.16782261 0.09275515 0.73942224]
1 [0.03792975 0.69679192 0.26527833]
1 [0.27726385 0.23361566 0.4891205 ]
2 [0.17774229 0.62555128 0.19670643]
2 [0.01997489 0.06799401 0.9120311 ]
2 [0.17548715 0.13428573 0.69022711]
2 [0.2181171  0.18397526 0.59790764]
2 [0.28757289 0.10431355 0.60811356]
1 [0.07861273 0.22124423 0.70014304]
0 [0.83479392 0.09915308 0.066053  ]


mean features:
layers=112
test_size=0.3
alpha=0.1
lambda=0.1

* * *
training accuracy: 100.0%
testing accuracy: 87.847

* * *
training accuracy: 100.0%
testing accuracy: 84.375%

Confusion Matrix: 
[[239  28   9]
 [ 26 249  21]
 [ 17  34 241]]

first 10 test values and predictions:
1 [0.00276413 0.96965361 0.02758227]
0 [9.99835185e-01 1.64640491e-04 1.74103521e-07]
0 [0.23397429 0.50756851 0.2584572 ]
0 [9.99947303e-01 5.26716685e-05 2.56000473e-08]
2 [3.75749914e-08 6.39833733e-06 9.99993564e-01]
2 [0.00170406 0.04003609 0.95825985]
0 [0.79005888 0.07589678 0.13404434]
0 [9.99968185e-01 3.10770078e-05 7.37597368e-07]
1 [0.00902078 0.62433826 0.36664095]
2 [0.01094249 0.02162663 0.96743089]


mean features:
layers=112
test_size=0.3
alpha=0.001
lambda=1000



mean features:
layers=112
test_size=0.3
alpha=0.001
lambda=100



mean features:
layers=112
test_size=0.3
alpha=0.001
lambda=1.0

* * *
training accuracy: 92.11309523809523%
testing accuracy: 82.63888888888889%

Confusion Matrix: 
[[238  27  22]
 [ 25 232  34]
 [ 12  30 244]]

first 10 test values and predictions:
0 [0.2637556  0.26804319 0.4682012

* * *
training accuracy: 100.0%
testing accuracy: 85.41666666666666%

Confusion Matrix: 
[[341  31  23]
 [ 33 311  25]
 [ 17  39 332]]

first 10 test values and predictions:
2 [1.04577198e-07 5.20650375e-06 9.99994689e-01]
0 [9.92012829e-01 7.97079941e-03 1.63718973e-05]
1 [3.29613457e-07 9.99999670e-01 4.14933344e-11]
2 [5.43003217e-08 8.58581084e-03 9.91414135e-01]
1 [0.0013221  0.94040879 0.05826911]
2 [6.42830953e-06 3.86312371e-05 9.99954940e-01]
1 [2.72787239e-02 9.72719961e-01 1.31497508e-06]
2 [1.05622489e-06 1.75754819e-05 9.99981368e-01]
1 [2.33586779e-04 9.99764811e-01 1.60227511e-06]
1 [7.84456249e-05 9.99901084e-01 2.04699759e-05]


mean features:
layers=(112, 112, 112)
test_size=0.4
alpha=0.1
lambda=0.0001

* * *
training accuracy: 100.0%
testing accuracy: 83.85416666666666%

Confusion Matrix: 
[[323  34  26]
 [ 39 320  30]
 [ 24  33 323]]

first 10 test values and predictions:
1 [1.38180887e-02 9.86181418e-01 4.93005658e-07]
0 [9.99828486e-01 1.71265603e-04 2.48645762e-0

* * *
training accuracy: 100.0%
testing accuracy: 82.29166666666666%

Confusion Matrix: 
[[328  32  26]
 [ 28 309  55]
 [ 23  40 311]]

first 10 test values and predictions:
0 [9.89586814e-01 1.54106964e-04 1.02590787e-02]
0 [9.99051468e-01 9.48508707e-04 2.32073728e-08]
0 [0.9966094  0.00104364 0.00234696]
0 [0.98622523 0.00169896 0.01207581]
2 [4.99502852e-04 1.79489435e-01 8.20011062e-01]
0 [0.51080146 0.26849706 0.22070148]
2 [1.86480870e-06 1.97531544e-05 9.99978382e-01]
1 [5.67796739e-04 9.77548916e-01 2.18832876e-02]
1 [7.66830728e-06 9.89336854e-01 1.06554780e-02]
0 [0.013572   0.04397959 0.94244841]


mean features:
layers=(112, 112, 112)
test_size=0.4
alpha=0.001
lambda=0.001

* * *
training accuracy: 100.0%
testing accuracy: 83.07291666666666%

Confusion Matrix: 
[[329  38  30]
 [ 27 308  22]
 [ 26  52 320]]

first 10 test values and predictions:
2 [0.03317242 0.00340316 0.96342443]
1 [1.86635533e-05 9.99078905e-01 9.02431850e-04]
0 [0.20438237 0.00659416 0.78902348]
0 [0.80

* * *
training accuracy: 100.0%
testing accuracy: 85.18518518518519%

Confusion Matrix: 
[[262  24  26]
 [ 14 235  27]
 [ 17  20 239]]

first 10 test values and predictions:
2 [0.41431746 0.10758345 0.47809909]
1 [0.23256814 0.76462573 0.00280612]
2 [9.39535657e-06 8.95410801e-01 1.04579804e-01]
1 [3.99175733e-03 9.95994168e-01 1.40748264e-05]
2 [0.55372691 0.00126469 0.44500841]
2 [1.21503276e-04 2.58575923e-03 9.97292737e-01]
0 [9.95727066e-01 4.04071441e-03 2.32220003e-04]
2 [1.01258921e-04 2.59113209e-07 9.99898482e-01]
2 [0.01982671 0.00204203 0.97813126]
2 [2.84335195e-06 4.59192494e-07 9.99996697e-01]


mean features:
layers=(112, 112, 112)
test_size=0.3
alpha=0.01
lambda=0.001

* * *
training accuracy: 100.0%
testing accuracy: 84.14351851851852%

Confusion Matrix: 
[[242  25  22]
 [ 21 220  25]
 [ 11  33 265]]

first 10 test values and predictions:
1 [1.45191012e-02 9.85445314e-01 3.55851450e-05]
0 [9.99999930e-01 6.51086327e-08 4.43493008e-09]
2 [6.07356873e-14 3.41257049e-04 

In [30]:
# looking at the top models
show_above(model_info_c_tanh, 0.88)

0.9114583333333334
	features: all features
	layers: 112
	test size: 0.4
	alpha: 1.0
	lambda: 0.1
	train accuracy: 1.0
	test accuracy: 0.9114583333333334
	classifier: MLPClassifier(activation='tanh', alpha=0.1, hidden_layer_sizes=112,
              learning_rate_init=1.0, max_iter=3000, random_state=1,
              solver='sgd')
0.9097222222222222
	features: all features
	layers: 112
	test size: 0.3
	alpha: 0.1
	lambda: 0.1
	train accuracy: 1.0
	test accuracy: 0.9097222222222222
	classifier: MLPClassifier(activation='tanh', alpha=0.1, hidden_layer_sizes=112,
              learning_rate_init=0.1, max_iter=3000, random_state=1,
              solver='sgd')
0.9085648148148148
	features: all features
	layers: (112, 56, 30, 10)
	test size: 0.3
	alpha: 0.001
	lambda: 1.0
	train accuracy: 1.0
	test accuracy: 0.9085648148148148
	classifier: MLPClassifier(activation='tanh', alpha=1.0,
              hidden_layer_sizes=(112, 56, 30, 10), max_iter=3000,
              random_state=1, solver='sgd')
0

### User D

In [31]:
model_info_d_log = {} # key is accuracy, values are 
automate_run(df2, model_info_d_log)



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=1.0
lambda=1000



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=1.0
lambda=100



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=1.0
lambda=1.0



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=1.0
lambda=0.1



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=1.0
lambda=0.01



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=1.0
lambda=0.001



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=1.0
lambda=0.0001



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=1.0
lambda=1e-05



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=0.1
lambda=1000



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=0.1
lambda=100



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=0.1
lambda=1.0



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=0.1
lambda=0.1



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=0.1
l

* * *
training accuracy: 100.0%
testing accuracy: 88.19444444444444%

Confusion Matrix: 
[[360  14  25]
 [ 21 316  35]
 [ 17  24 340]]

first 10 test values and predictions:
2 [2.89893680e-07 1.45610224e-08 9.99999696e-01]
2 [1.25654256e-08 6.81030593e-07 9.99999306e-01]
0 [9.99811119e-01 1.88666735e-04 2.13856883e-07]
2 [3.50848257e-04 1.68339295e-03 9.97965759e-01]
2 [1.07672167e-12 5.10137896e-10 9.99999999e-01]
2 [0.0148369  0.03820777 0.94695533]
0 [4.46133776e-01 1.03595318e-04 5.53762628e-01]
0 [9.99948468e-01 3.78813587e-05 1.36503872e-05]
1 [4.85275091e-07 9.84754537e-01 1.52449776e-02]
0 [9.99551926e-01 4.83150900e-05 3.99759159e-04]


all features:
layers=112
test_size=0.4
alpha=1.0
lambda=0.001

* * *
training accuracy: 100.0%
testing accuracy: 87.67361111111111%

Confusion Matrix: 
[[344  18  26]
 [ 25 322  21]
 [ 25  27 344]]

first 10 test values and predictions:
0 [9.99942619e-01 5.73809545e-05 7.24813512e-13]
2 [1.08972262e-16 6.42077000e-05 9.99935792e-01]
0 [9.934230

* * *
training accuracy: 100.0%
testing accuracy: 87.23958333333334%

Confusion Matrix: 
[[338  20  30]
 [ 33 318  35]
 [ 12  17 349]]

first 10 test values and predictions:
2 [1.68006277e-03 6.81155507e-04 9.97638782e-01]
2 [0.00111405 0.00243181 0.99645414]
2 [1.05651996e-02 3.27989663e-04 9.89106811e-01]
1 [0.32432343 0.07788484 0.59779173]
1 [0.01084251 0.98521021 0.00394728]
0 [0.92365812 0.05870737 0.01763451]
1 [0.80760189 0.16657198 0.02582613]
2 [0.19858188 0.21380349 0.58761463]
1 [8.67407545e-04 9.81053162e-01 1.80794308e-02]
2 [0.00126617 0.02882277 0.96991107]


all features:
layers=112
test_size=0.4
alpha=0.01
lambda=0.01

* * *
training accuracy: 100.0%
testing accuracy: 87.06597222222221%

Confusion Matrix: 
[[344  27  18]
 [ 19 327  36]
 [ 25  24 332]]

first 10 test values and predictions:
1 [0.00113813 0.9977696  0.00109227]
2 [0.99622627 0.00118597 0.00258776]
1 [6.76670935e-06 9.99858074e-01 1.35158803e-04]
1 [1.91567422e-06 9.99862714e-01 1.35370066e-04]
1 [5.6501

C:\Users\Elijah\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:684: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (3000) reached and the optimization hasn't converged yet.
  warnings.warn(




all features:
layers=112
test_size=0.4
alpha=0.001
lambda=0.01



...

* * *
training accuracy: 100.0%
testing accuracy: 84.20138888888889%

Confusion Matrix: 
[[338  26  30]
 [ 28 309  42]
 [ 14  42 323]]

first 10 test values and predictions:
2 [1.56294846e-05 9.89210610e-01 1.07737602e-02]
0 [9.99966611e-01 6.72564008e-09 3.33826366e-05]
2 [0.00702047 0.91503111 0.07794842]
0 [0.66393262 0.05002784 0.28603954]
0 [9.98165215e-01 1.66831578e-03 1.66469562e-04]
1 [2.86082930e-07 8.46864921e-04 9.99152849e-01]
0 [9.73468153e-01 2.61961701e-02 3.35676778e-04]
2 [1.64120160e-07 3.49610980e-04 9.99650225e-01]
0 [9.99999822e-01 1.76310844e-07 1.27949226e-09]
1 [1.09283201e-03 9.98892601e-01 1.45665694e-05]


mean features:
layers=112
test_size=0.4
alpha=1.0
lambda=0.0001

* * *
training accuracy: 100.0%
testing accuracy: 81.68402777777779%

Confusion Matrix: 
[[325  43  27]
 [ 28 309  43]
 [ 15  55 307]]

first 10 test values and predictions:
1 [4.98692693e-04 9.82158281e-01 1.73430265e-02]
2 [9.54313917e-05 5.65723492e-01 4.34181077e-01]
0 [9.99938747e-01 6.1

* * *
training accuracy: 100.0%
testing accuracy: 84.02777777777779%

Confusion Matrix: 
[[346  47  16]
 [ 24 296  36]
 [ 26  35 326]]

first 10 test values and predictions:
0 [0.19804961 0.78143893 0.02051146]
0 [0.73569811 0.25643062 0.00787128]
1 [0.01379528 0.98151685 0.00468786]
2 [2.78780678e-06 1.07741941e-04 9.99889470e-01]
0 [9.98619383e-01 1.06582640e-03 3.14790812e-04]
1 [0.03903121 0.60320152 0.35776727]
1 [6.95454180e-05 9.81720982e-01 1.82094729e-02]
1 [0.99160851 0.00180537 0.00658612]
2 [0.00115463 0.01747835 0.98136703]
1 [2.72761460e-04 7.22423882e-01 2.77303356e-01]


mean features:
layers=112
test_size=0.4
alpha=0.01
lambda=1e-05

* * *
training accuracy: 100.0%
testing accuracy: 81.59722222222221%

Confusion Matrix: 
[[318  36  29]
 [ 42 305  31]
 [ 21  53 317]]

first 10 test values and predictions:
1 [7.48087021e-08 9.99995960e-01 3.96531783e-06]
0 [9.97208532e-01 2.30958180e-05 2.76837235e-03]
0 [0.92893461 0.00116672 0.06989867]
2 [3.39171712e-10 1.11803918e-07

* * *
training accuracy: 100.0%
testing accuracy: 86.80555555555556%

Confusion Matrix: 
[[263  18  10]
 [ 22 235  25]
 [ 14  25 252]]

first 10 test values and predictions:
2 [0.00448716 0.00488607 0.99062677]
2 [5.37957424e-05 1.62208292e-03 9.98324121e-01]
2 [7.00278628e-05 4.17815419e-04 9.99512157e-01]
2 [6.82691027e-10 4.24912288e-05 9.99957508e-01]
2 [1.17800960e-05 2.69775334e-09 9.99988217e-01]
2 [0.00618076 0.14885007 0.84496917]
0 [9.99824753e-01 1.43081427e-04 3.21654654e-05]
0 [9.98562396e-01 1.32928030e-06 1.43627471e-03]
2 [2.51423717e-06 1.00014891e-06 9.99996486e-01]
2 [8.74497902e-04 3.36050622e-02 9.65520440e-01]


mean features:
layers=112
test_size=0.3
alpha=0.1
lambda=1e-05

* * *
training accuracy: 100.0%
testing accuracy: 85.06944444444444%

Confusion Matrix: 
[[249  23  19]
 [ 20 234  28]
 [ 16  23 252]]

first 10 test values and predictions:
2 [4.69517276e-04 8.45632353e-04 9.98684850e-01]
1 [8.16882588e-03 9.91830938e-01 2.36026996e-07]
2 [0.90672437 0.083975

* * *
training accuracy: 100.0%
testing accuracy: 83.94097222222221%

Confusion Matrix: 
[[309  41  31]
 [ 22 337  35]
 [ 16  40 321]]

first 10 test values and predictions:
2 [1.38920625e-08 1.52900156e-07 9.99999833e-01]
1 [1.02539022e-10 9.99999970e-01 3.01479594e-08]
1 [5.18319755e-05 9.99865469e-01 8.26990604e-05]
1 [4.15065753e-07 9.99997601e-01 1.98424118e-06]
1 [8.01339635e-10 9.99975153e-01 2.48457710e-05]
0 [2.36863589e-01 7.63136399e-01 1.21126699e-08]
2 [0.02806509 0.68695043 0.28498447]
0 [2.15373766e-05 9.99976823e-01 1.63997436e-06]
1 [1.03453888e-04 9.99896429e-01 1.16822184e-07]
1 [1.04556772e-05 9.99989098e-01 4.46817741e-07]


mean features:
layers=(112, 112, 112)
test_size=0.4
alpha=0.1
lambda=1e-05

* * *
training accuracy: 100.0%
testing accuracy: 82.8125%

Confusion Matrix: 
[[325  34  21]
 [ 45 312  31]
 [ 26  41 317]]

first 10 test values and predictions:
0 [9.99619623e-01 3.80376701e-04 6.90430062e-12]
2 [1.09838154e-17 2.54006330e-11 1.00000000e+00]
1 [0.001



mean features:
layers=(112, 112, 112)
test_size=0.3
alpha=0.01
lambda=1e-05



mean features:
layers=(112, 112, 112)
test_size=0.3
alpha=0.001
lambda=1000



mean features:
layers=(112, 112, 112)
test_size=0.3
alpha=0.001
lambda=100



mean features:
layers=(112, 112, 112)
test_size=0.3
alpha=0.001
lambda=1.0



mean features:
layers=(112, 112, 112)
test_size=0.3
alpha=0.001
lambda=0.1



mean features:
layers=(112, 112, 112)
test_size=0.3
alpha=0.001
lambda=0.01



mean features:
layers=(112, 112, 112)
test_size=0.3
alpha=0.001
lambda=0.001



mean features:
layers=(112, 112, 112)
test_size=0.3
alpha=0.001
lambda=0.0001



mean features:
layers=(112, 112, 112)
test_size=0.3
alpha=0.001
lambda=1e-05



In [32]:
# looking at the top models
show_above(model_info_d_log, 0.88)

0.9050925925925926
	features: all features
	layers: 112
	test size: 0.3
	alpha: 1.0
	lambda: 0.01
	train accuracy: 1.0
	test accuracy: 0.9050925925925926
	classifier: MLPClassifier(activation='logistic', alpha=0.01, hidden_layer_sizes=112,
              learning_rate_init=1.0, max_iter=3000, random_state=1,
              solver='sgd')
0.9016203703703703
	features: all features
	layers: 112
	test size: 0.3
	alpha: 1.0
	lambda: 0.001
	train accuracy: 1.0
	test accuracy: 0.9016203703703703
	classifier: MLPClassifier(activation='logistic', alpha=0.001, hidden_layer_sizes=112,
              learning_rate_init=1.0, max_iter=3000, random_state=1,
              solver='sgd')
0.8993055555555556
	features: all features
	layers: 112
	test size: 0.3
	alpha: 0.01
	lambda: 0.1
	train accuracy: 1.0
	test accuracy: 0.8993055555555556
	classifier: MLPClassifier(activation='logistic', alpha=0.1, hidden_layer_sizes=112,
              learning_rate_init=0.01, max_iter=3000, random_state=1,
              s

In [33]:
model_info_d_tanh = {} # key is accuracy, values are 
automate_run(df2, model_info_d_tanh, act='tanh')



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=1.0
lambda=1000



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=1.0
lambda=100



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=1.0
lambda=1.0



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=1.0
lambda=0.1



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=1.0
lambda=0.01



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=1.0
lambda=0.001



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=1.0
lambda=0.0001



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=1.0
lambda=1e-05



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=0.1
lambda=1000



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=0.1
lambda=100



all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=0.1
lambda=1.0

* * *
training accuracy: 99.82638888888889%
testing accuracy: 88.28125%

Confusion Matrix: 
[[360  13  20]
 [ 18 308  49]
 [ 10  25 349]]

firs

* * *
training accuracy: 100.0%
testing accuracy: 81.25%

Confusion Matrix: 
[[324  28  30]
 [ 39 312  52]
 [ 29  38 300]]

first 10 test values and predictions:
1 [0.00138676 0.9677327  0.03088054]
0 [9.96568112e-01 3.36172650e-03 7.01616424e-05]
0 [9.99826399e-01 8.35877487e-05 9.00136522e-05]
0 [0.97906265 0.00554159 0.01539577]
1 [0.04179154 0.43118585 0.52702261]
0 [9.99544453e-01 7.50360006e-05 3.80510963e-04]
0 [9.98251300e-01 1.72746226e-03 2.12380113e-05]
0 [9.94817061e-01 1.34028960e-05 5.16953605e-03]
0 [9.92304792e-01 7.66369816e-03 3.15096078e-05]
2 [0.00311638 0.40658447 0.59029915]


all features:
layers=(112, 56, 30, 10)
test_size=0.4
alpha=0.01
lambda=1e-05

* * *
training accuracy: 100.0%
testing accuracy: 83.85416666666666%

Confusion Matrix: 
[[335  23  33]
 [ 19 318  46]
 [ 34  31 313]]

first 10 test values and predictions:
2 [0.00503708 0.00259403 0.99236889]
2 [4.75309959e-01 3.85348530e-04 5.24304692e-01]
2 [0.0178012  0.30424777 0.67795103]
2 [0.40942616 0.026

* * *
training accuracy: 100.0%
testing accuracy: 87.5%

Confusion Matrix: 
[[256  15  12]
 [ 22 259  26]
 [ 16  17 241]]

first 10 test values and predictions:
0 [9.91616486e-01 3.41664167e-06 8.38009746e-03]
1 [6.11762844e-04 9.99272673e-01 1.15563902e-04]
0 [9.90643960e-01 9.99085548e-07 9.35504125e-03]
2 [0.02760524 0.00334451 0.96905025]
0 [9.96591867e-01 1.93378443e-06 3.40619880e-03]
1 [4.85804183e-06 9.99767440e-01 2.27701649e-04]
1 [9.80614687e-01 1.93791495e-02 6.16367663e-06]
2 [1.30846728e-05 1.02870098e-04 9.99884045e-01]
1 [3.34549677e-06 9.99387524e-01 6.09130979e-04]
0 [9.99713856e-01 2.85170908e-04 9.73029498e-07]


all features:
layers=(112, 56, 30, 10)
test_size=0.3
alpha=0.1
lambda=1e-05

* * *
training accuracy: 100.0%
testing accuracy: 86.68981481481481%

Confusion Matrix: 
[[241  11  15]
 [ 17 265  29]
 [ 14  29 243]]

first 10 test values and predictions:
0 [9.99245750e-01 2.31418307e-05 7.31108032e-04]
0 [9.99654914e-01 3.44915330e-04 1.70679393e-07]
0 [9.91203



all features:
layers=(112, 56, 30, 10)
test_size=0.3
alpha=0.001
lambda=0.0001



all features:
layers=(112, 56, 30, 10)
test_size=0.3
alpha=0.001
lambda=1e-05

* * *
training accuracy: 100.0%
testing accuracy: 83.33333333333334%

Confusion Matrix: 
[[229  19  24]
 [ 14 256  23]
 [ 25  39 235]]

first 10 test values and predictions:
0 [9.98818475e-01 9.73610613e-04 2.07914765e-04]
0 [9.88309468e-01 1.31040049e-04 1.15594915e-02]
0 [9.98363160e-01 1.37538311e-03 2.61457217e-04]
0 [9.85142302e-01 3.82476826e-04 1.44752212e-02]
0 [9.90198465e-01 1.86356365e-04 9.61517827e-03]
2 [0.01940298 0.02833165 0.95226537]
1 [0.39949657 0.598854   0.00164943]
1 [6.23209945e-04 9.96401289e-01 2.97550070e-03]
2 [0.05852828 0.01225295 0.92921877]
1 [1.41900153e-04 9.94549036e-01 5.30906337e-03]


all features:
layers=112
test_size=0.4
alpha=1.0
lambda=1000



all features:
layers=112
test_size=0.4
alpha=1.0
lambda=100



all features:
layers=112
test_size=0.4
alpha=1.0
lambda=1.0



all features:
lay

* * *
training accuracy: 100.0%
testing accuracy: 86.11111111111111%

Confusion Matrix: 
[[358  29  26]
 [ 19 336  31]
 [ 21  34 298]]

first 10 test values and predictions:
2 [4.84327944e-04 1.04201464e-04 9.99411471e-01]
2 [0.00160604 0.85772118 0.14067278]
0 [0.48217447 0.01060274 0.5072228 ]
2 [0.0648047 0.0134022 0.9217931]
2 [3.71133525e-05 5.11715681e-04 9.99451171e-01]
0 [9.97445320e-01 3.12938646e-05 2.52338627e-03]
0 [9.89685903e-01 9.35972221e-03 9.54375278e-04]
1 [0.15363218 0.77217876 0.07418907]
0 [9.97356532e-01 1.64893576e-07 2.64330357e-03]
1 [2.52241167e-03 9.97426505e-01 5.10837799e-05]


all features:
layers=112
test_size=0.4
alpha=0.01
lambda=1000



all features:
layers=112
test_size=0.4
alpha=0.01
lambda=100



all features:
layers=112
test_size=0.4
alpha=0.01
lambda=1.0

* * *
training accuracy: 100.0%
testing accuracy: 88.62847222222221%

Confusion Matrix: 
[[346  28   6]
 [ 18 327  36]
 [ 18  25 348]]

first 10 test values and predictions:
1 [0.02112576 0.9653

...

* * *
training accuracy: 100.0%
testing accuracy: 85.64814814814815%

Confusion Matrix: 
[[247  16   7]
 [ 34 242  20]
 [ 13  34 251]]

first 10 test values and predictions:
2 [0.00257702 0.04422917 0.95319381]
1 [0.63756031 0.36023015 0.00220954]
0 [9.97980599e-01 1.98444922e-03 3.49517863e-05]
1 [1.90628295e-01 8.09108020e-01 2.63684935e-04]
1 [0.00177818 0.11695609 0.88126573]
1 [0.99109828 0.0074763  0.00142543]
0 [9.98429861e-01 1.56239873e-03 7.73983281e-06]
2 [8.06152101e-05 2.22563532e-03 9.97693749e-01]
1 [0.73845315 0.2477878  0.01375905]
0 [0.82973656 0.16033898 0.00992446]


mean features:
layers=112
test_size=0.3
alpha=0.1
lambda=0.001

* * *
training accuracy: 100.0%
testing accuracy: 86.92129629629629%

Confusion Matrix: 
[[258  20   6]
 [ 26 244  17]
 [ 16  28 249]]

first 10 test values and predictions:
0 [9.99869804e-01 1.64338697e-05 1.13762290e-04]
0 [9.99997039e-01 7.15941309e-07 2.24464372e-06]
1 [0.07568888 0.92044153 0.00386959]
0 [9.46563042e-01 5.30780486e-02 

* * *
training accuracy: 99.95039682539682%
testing accuracy: 86.3425925925926%

Confusion Matrix: 
[[250  20  20]
 [ 17 254  20]
 [ 16  25 242]]

first 10 test values and predictions:
1 [1.60540826e-04 9.83958736e-01 1.58807228e-02]
2 [0.00677107 0.74148007 0.25174885]
0 [0.92410795 0.03364876 0.04224329]
0 [0.98625759 0.00299099 0.01075142]
1 [0.08251583 0.69862642 0.21885775]
2 [0.00200703 0.05040423 0.94758874]
0 [9.92137498e-01 7.63262704e-03 2.29874491e-04]
1 [0.0107329  0.91032857 0.07893853]
0 [9.95003715e-01 4.88630131e-03 1.09983966e-04]
2 [0.12374778 0.00953679 0.86671543]


mean features:
layers=112
test_size=0.3
alpha=0.001
lambda=0.001

* * *
training accuracy: 99.95039682539682%
testing accuracy: 86.11111111111111%

Confusion Matrix: 
[[259  21  24]
 [ 24 228  23]
 [  9  19 257]]

first 10 test values and predictions:
1 [0.18811893 0.79601862 0.01586245]
2 [0.00360188 0.11190675 0.88449137]
1 [9.25197946e-06 9.99990572e-01 1.75767286e-07]
0 [0.79730919 0.03512509 0.16756



mean features:
layers=(112, 112, 112)
test_size=0.4
alpha=0.01
lambda=100



mean features:
layers=(112, 112, 112)
test_size=0.4
alpha=0.01
lambda=1.0

* * *
training accuracy: 100.0%
testing accuracy: 87.93402777777779%

Confusion Matrix: 
[[354  23  13]
 [ 29 332  25]
 [ 18  31 327]]

first 10 test values and predictions:
2 [0.0901376  0.50353222 0.40633018]
2 [0.03841766 0.24035267 0.72122967]
2 [0.01115626 0.31180845 0.67703529]
2 [0.68494242 0.03025994 0.28479764]
0 [0.9684553  0.00852437 0.02302033]
1 [0.00385211 0.9944084  0.00173949]
1 [0.00146817 0.99131866 0.00721317]
0 [9.99597657e-01 3.84299580e-04 1.80431142e-05]
2 [5.86753791e-01 1.38767706e-04 4.13107441e-01]
0 [9.94211652e-01 1.68476494e-04 5.61987118e-03]


mean features:
layers=(112, 112, 112)
test_size=0.4
alpha=0.01
lambda=0.1

* * *
training accuracy: 100.0%
testing accuracy: 82.46527777777779%

Confusion Matrix: 
[[301  35  33]
 [ 42 332  40]
 [ 17  35 317]]

first 10 test values and predictions:
0 [1.70623973e-

* * *
training accuracy: 100.0%
testing accuracy: 82.20486111111111%

Confusion Matrix: 
[[316  36  25]
 [ 32 311  37]
 [ 34  41 320]]

first 10 test values and predictions:
0 [9.99992638e-01 5.71847328e-06 1.64303201e-06]
1 [5.47737033e-05 9.99945113e-01 1.13124708e-07]
2 [1.98593901e-04 3.99380768e-05 9.99761468e-01]
1 [0.02017404 0.90586421 0.07396175]
0 [9.84110179e-01 1.58548612e-02 3.49602097e-05]
2 [0.21558101 0.00081096 0.78360803]
2 [5.98820985e-03 6.55811436e-05 9.93946209e-01]
0 [9.93135427e-01 6.86059083e-03 3.98247039e-06]
0 [9.88742973e-01 1.12565342e-02 4.93272273e-07]
2 [5.83027715e-05 2.19700747e-01 7.80240951e-01]


mean features:
layers=(112, 112, 112)
test_size=0.3
alpha=1.0
lambda=1000



mean features:
layers=(112, 112, 112)
test_size=0.3
alpha=1.0
lambda=100



mean features:
layers=(112, 112, 112)
test_size=0.3
alpha=1.0
lambda=1.0



mean features:
layers=(112, 112, 112)
test_size=0.3
alpha=1.0
lambda=0.1



mean features:
layers=(112, 112, 112)
test_size=0.3
a

* * *
training accuracy: 100.0%
testing accuracy: 86.45833333333334%

Confusion Matrix: 
[[239  21   8]
 [ 25 244  16]
 [ 23  24 264]]

first 10 test values and predictions:
1 [9.97705270e-01 2.19843145e-03 9.62981595e-05]
0 [0.32020188 0.66950958 0.01028855]
2 [1.43741579e-04 9.99340938e-01 5.15320450e-04]
2 [2.11934865e-05 3.04402202e-05 9.99948366e-01]
1 [3.02658816e-04 9.99670777e-01 2.65638558e-05]
1 [2.41184865e-07 9.96542578e-01 3.45718106e-03]
2 [0.00291714 0.93024352 0.06683934]
1 [2.01463983e-04 9.01288546e-01 9.85099905e-02]
2 [3.02409499e-03 5.51479340e-06 9.96970390e-01]
2 [6.34629058e-05 8.22935078e-01 1.77001459e-01]


mean features:
layers=(112, 112, 112)
test_size=0.3
alpha=0.01
lambda=1e-05

* * *
training accuracy: 100.0%
testing accuracy: 85.5324074074074%

Confusion Matrix: 
[[256  28  23]
 [ 16 236  22]
 [ 16  20 247]]

first 10 test values and predictions:
0 [9.99791112e-01 2.48298874e-08 2.08863306e-04]
1 [2.73520821e-04 3.37645186e-01 6.62081294e-01]
1 [6.41897

In [34]:
# looking at the top models
show_above(model_info_d_tanh, 0.88)

0.9175347222222222
	features: all features
	layers: 112
	test size: 0.4
	alpha: 1.0
	lambda: 0.1
	train accuracy: 1.0
	test accuracy: 0.9175347222222222
	classifier: MLPClassifier(activation='tanh', alpha=0.1, hidden_layer_sizes=112,
              learning_rate_init=1.0, max_iter=3000, random_state=1,
              solver='sgd')
0.9097222222222222
	features: all features
	layers: (112, 56, 30, 10)
	test size: 0.3
	alpha: 0.001
	lambda: 1.0
	train accuracy: 1.0
	test accuracy: 0.9097222222222222
	classifier: MLPClassifier(activation='tanh', alpha=1.0,
              hidden_layer_sizes=(112, 56, 30, 10), max_iter=3000,
              random_state=1, solver='sgd')
0.9016203703703703
	features: all features
	layers: (112, 112, 112)
	test size: 0.3
	alpha: 0.01
	lambda: 1.0
	train accuracy: 1.0
	test accuracy: 0.9016203703703703
	classifier: MLPClassifier(activation='tanh', alpha=1.0, hidden_layer_sizes=(112, 112, 112),
              learning_rate_init=0.01, max_iter=3000, random_state=1,
   

## Saving the data

In [35]:
models = [
    model_info_a_log, 
    model_info_b_log,
    model_info_c_log,
    model_info_d_log,
    
    model_info_a_tanh,
    model_info_b_tanh,
    model_info_c_tanh,
    model_info_d_tanh 
]

In [7]:
# to save the model info trained here
# NOTE: this is not to save numpy arrays. If we want to do that, use np.save and np.load.
import pickle

models_file = open('models', 'wb')

pickle.dump(models, models_file)

models_file.close()

In [37]:
# to load data into models again
import pickle

models_file = open('models', 'rb')

models_loaded = pickle.load(models_file)

models_file.close()

In [40]:
models[0][1]

{'features': 'all features',
 'layers': (112, 56, 30, 10),
 'test size': 0.4,
 'alpha': 1.0,
 'lambda': 100,
 'train accuracy': 0.34664351851851855,
 'test accuracy': 0.3133680555555556,
 'classifier': MLPClassifier(activation='logistic', alpha=100,
               hidden_layer_sizes=(112, 56, 30, 10), learning_rate_init=1.0,
               max_iter=3000, random_state=1, solver='sgd')}

In [41]:
models_loaded[0][1]

{'features': 'all features',
 'layers': (112, 56, 30, 10),
 'test size': 0.4,
 'alpha': 1.0,
 'lambda': 100,
 'train accuracy': 0.34664351851851855,
 'test accuracy': 0.3133680555555556,
 'classifier': MLPClassifier(activation='logistic', alpha=100,
               hidden_layer_sizes=(112, 56, 30, 10), learning_rate_init=1.0,
               max_iter=3000, random_state=1, solver='sgd')}